# RLHF From Scratch on LLMs

In this notebook, I will start with history of RLHF, the importance of RLHF in LLMs, then go into the architectures TRPO, PPO, GRPO and DPO. Each of the technique's explanation will have the math, code and explanations on how it's done, finally in the end we'll experiment these techniques on one of prebuilt LLMs (the llms are not built from scratch, since i've already done that in my [llm-from-scratch repository](https://github.com/ashworks1706/llm-from-scratch)). If you're new to RL, check out [dqn-from-scratch](https://github.com/ashworks1706/dqn-from-scratch) where i've explained RL indepth from the core

## Brief History of RLHF



RLHF emerged around 2017-2018 when researchers at OpenAI developed techniques to incorporate human preferences into reinforcement learning systems. The seminal paper "Deep Reinforcement Learning from Human Preferences" by Christiano et al. (2017) introduced the core concept of using human comparisons between pairs of outputs to train a reward model that could guide RL agents toward preferred behaviors. While initially applied to simpler tasks and robotics, the technique remained relatively specialized until recent years. The technique gained mainstream attention in 2022 when OpenAI used it to create ChatGPT from GPT-3.5, dramatically improving output quality by aligning the model with human preferences. This breakthrough demonstrated RLHF's potential to transform raw language model capabilities into systems that better align with human intent and values. Since then, RLHF has become a standard component in developing advanced language models like GPT-4, Claude, and Llama 2, with each iteration refining the techniques to achieve better alignment.

#### Why RLHF Matters for LLMs

<img src="assets/rlhf-vs-finetune.png" width=300>

Large Language Models trained solely on next-token prediction are just models with knowledge, they don't know how to answer properly. You have model trained on shakespear work, great! but how do you make it to answer questions in the way we want (in the way humans talk)? These models optimize for predicting the next token based on training data distribution, which doesn't necessarily correlate with producing helpful, harmless, or honest responses. Traditional LLMs may generate toxic, harmful, or misleading content because they're simply trying to produce statistically likely continuations without understanding human values or preferences. They lack an inherent mechanism to distinguish between content that is statistically probable and content that is actually desirable according to human standards. RLHF addresses these issues by creating a feedback loop where human preferences explicitly guide the model's learning process, steering it toward outputs that humans find more helpful, honest, and aligned with their intent. This alignment process transforms a powerful but directionless prediction engine into a system that can better understand and respect nuanced human values and follow complex instructions in ways that maximize utility for users.




## The Birds Eye View

<img src="assets/workflow.png" >


Before delving into the complexities of RLHF, it's essential to understand the overal workflow of what actually happens in a typical RLHF based projects. When a large language model is initially trained on vast internet text corpora, it develops remarkable capabilities to predict text and acquire factual knowledge, but this training alone doesn't prepare it to be helpful in specific contexts or respond appropriately to human instructions. Consider a language model trained on extensive educational materials, including university canvas modules, academic papers, and textbooks. This model would possess substantial knowledge about various academic subjects, pedagogical approaches, and educational concepts. However, if asked to "Explain the concept of photosynthesis to a 10-year-old," it might produce a technically accurate but overly complex explanation filled with academic jargon that would confuse rather than enlighten a young student. The model hasn't been optimized to serve as an effective tutor - it simply predicts what text might follow in educational materials. 

The Supervised Fine-Tuning stage addresses this gap by training the model on demonstrations of desired behavior. For our hypothetical educational assistant, SFT would involve collecting thousands of examples showing how skilled human tutors respond to student questions: simplifying complex concepts, using age-appropriate language, providing relevant examples, checking for understanding, and offering encouragement. These demonstrations are formatted as input-output pairs (prompt and ideal response), and the model is fine-tuned to minimize the difference between its outputs and these human-generated "gold standard" responses. Through this process, the model learns the patterns that characterize helpful tutoring: breaking down complex concepts into simpler components, using analogies relevant to younger audiences, avoiding unnecessary technical terms, and adopting a supportive tone. After SFT, when asked to explain photosynthesis to a 10-year-old, the model is much more likely to respond with an explanation involving plants "eating sunlight" and "breathing in carbon dioxide to make food," rather than discussing electron transport chains and ATP synthesis. The model hasn't gained new knowledge, but it has learned a new way to present its existing knowledge that better aligns with the specific goal of being an effective tutor for younger students. However, SFT alone has significant limitations. First, it can only learn from the specific examples it's shown, leaving gaps in how to handle the infinite variety of possible user requests. Second, the demonstrations might not cover the full range of desirable behaviors or edge cases where special handling is needed. Third, the quality of the SFT model depends entirely on the quality and consistency of the demonstration data. Finally, there's no mechanism for the model to understand why certain responses are better than others - it simply learns to mimic patterns without a deeper understanding of the preferences that make one response superior. These limitations are precisely what RLHF is designed to address in the subsequent stages of the alignment process.

Following Supervised Fine-Tuning, the RLHF workflow progresses to Human Preference Collection - a crucial stage that fundamentally changes how model improvement occurs. In this phase, rather than providing gold-standard demonstrations, human evaluators compare and rank different model responses to the same prompt. For our educational assistant, this might involve presenting evaluators with pairs of explanations for the same scientific concept and asking them which better achieves the goal of teaching a young student. One explanation might be more engaging and use more appropriate analogies, while another might be technically accurate but still too complex. By explicitly choosing the better response, humans provide preference signals that capture nuanced quality distinctions beyond what demonstration data alone can convey. These comparisons generate valuable datasets where each entry contains a prompt and two responses, with a label indicating which response humans preferred. The collection process typically gathers thousands or even millions of such comparative judgments, creating a rich dataset that embodies human preferences about what constitutes a high-quality response across diverse scenarios.

The third stage, Reward Model Training, transforms these human preferences into a quantifiable reward function that can guide further optimization. This reward model takes a prompt and response as input and outputs a scalar score representing how well the response aligns with human preferences. Technically, it's trained to predict which of two responses humans would prefer by maximizing the likelihood of the observed preference data. For our educational tutor, the reward model learns to assign higher scores to explanations that successfully simplify complex concepts without sacrificing accuracy, use age-appropriate analogies, maintain an encouraging tone, and check for understanding. This model becomes a computational proxy for human judgment, capable of evaluating millions of potential responses far beyond what human evaluators could manually assess. The quality of this reward model is critical, as it effectively defines what "good" means for all subsequent optimization.

With a trained reward model in place, the final stage applies Reinforcement Learning techniques to optimize the language model toward maximizing the predicted reward. The most common approach is Proximal Policy Optimization (PPO), which iteratively improves the model by adjusting its parameters to generate responses that receive higher reward scores. However, simply maximizing reward can lead to degenerate outputs that exploit loopholes in the reward model or diverge too far from natural language patterns. To prevent this, the optimization includes a "KL divergence" penalty that constrains how much the optimized model can deviate from the SFT model, preserving fluency and knowledge while improving alignment. For our educational tutor, this process might result in a model that maintains scientific accuracy while consistently finding creative, age-appropriate analogies and explanations across a much broader range of topics than were covered in the original demonstration data. The entire RLHF pipeline is often iterative, with new preference data collected from the improved model, leading to refined reward models and further optimization cycles. This continuous feedback loop progressively aligns the language model with human values and preferences, addressing the fundamental limitations of training on prediction alone or even on demonstration data without comparative preference signals.


## 1. Getting a pretrained LLM

<img src="assets/workflow1.png">


Now the first step is to have a fresh pretrained LLM right off the top. We'll be using huggingface library transformers library for our transformer components

In [ ]:
# %pip install transformers peft datasets tqdm wandb rouge-score
# PEFT is a technique to fine tune LLMs without modifying all of their parameters. it's efficient for our tutorial.

In [ ]:
# Import from library and setup the model class 
 
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel, PeftConfig
import os

class PretrainedLLM:
    def __init__(self, model_name="facebook/opt-350m", device=None):
        """
        Initializing a (No SFT RLHF) pretrained language model for RLHF experiment
        
        Args:
            model_name: HuggingFace model identifier (default: OPT-350M, a relatively small but capable model)
            device: Computing device (will auto-detect if None)
        """
        self.model_name = model_name
        
        # this code is just for detecting if you have Nvidia CUDA driver or not
        if device is None:
            self.device = "cuda" if torch.cuda.is_available() else "cpu"
        else:
            self.device = device
            
        print(f"Loading {model_name} on {self.device}...")
        
        # Load model and tokenizer (for full guide on implementing llm from scratch check out https://github.com/ashworks1706/llm-from-scratch
        self.tokenizer = AutoTokenizer.from_pretrained(model_name) 
        self.model = AutoModelForCausalLM.from_pretrained(
            model_name, 
            torch_dtype=torch.float16 if self.device == "cuda" else torch.float32,
            low_cpu_mem_usage=True
        )
        
        # distributed training for better GPU utilization
        self.model.to(self.device)
        print(f"Model loaded successfully with {sum(p.numel() for p in self.model.parameters())/1e6:.1f}M parameters")
        
    def generate(self, prompt, max_new_tokens=100, temperature=0.7, top_p=0.9):
        """
        Generate text from the model given a prompt (no RLHF)
        
        Args:
            prompt: Input text to generate from
            max_new_tokens: Maximum number of tokens to generate
            temperature: Sampling temperature (lower = more deterministic)
            top_p: Nucleus sampling parameter (lower = more focused)
            
        Returns:
            Generated text as string
        """
        inputs = self.tokenizer(prompt, return_tensors="pt").to(self.device)
        
        # Generate with sampling, no fancy tuning required
        with torch.no_grad():
            outputs = self.model.generate(
                **inputs,
                max_new_tokens=max_new_tokens,
                temperature=temperature,
                top_p=top_p,
                do_sample=True
            )
        
        # Decode and remove the prompt from the generated text
        full_text = self.tokenizer.decode(outputs[0], skip_special_tokens=True)
        generated_text = full_text[len(self.tokenizer.decode(inputs.input_ids[0], skip_special_tokens=True)):]
        
        return generated_text
    
    def save_checkpoint(self, path):
        """Save model checkpoint to the specified path"""
        self.model.save_pretrained(path)
        self.tokenizer.save_pretrained(path)
        print(f"Model saved to {path}")
        
    def load_adapter(self, adapter_path):
        """Load a PEFT adapter for efficient fine-tuning"""
        self.model = PeftModel.from_pretrained(
            self.model,
            adapter_path,
            torch_dtype=torch.float16 if self.device == "cuda" else torch.float32
        )
        print(f"Loaded adapter from {adapter_path}")



In [ ]:
# Example usage
if __name__ == "__main__":
    # Initialize with a small model for experimentation
    llm = PretrainedLLM(model_name="facebook/opt-350m")
    
    # Test generation
    prompt = "Explain quantum computing to a 10-year-old:"
    print(f"(No SFT RLHF) Prompt: {prompt}")
    print(f"(NO SFT RLHF) Response: {llm.generate(prompt, max_new_tokens=150)}")

---
(No SFT RLHF) *Prompt*: Explain quantum computing to a 10-year-old:

(No SFT RLHF) *Response*: Quantum computing is a type of computation that harnesses the collective properties of quantum states, such as superposition, interference, and entanglement, to perform calculations. The basic unit of quantum computation is the quantum bit or qubit. Unlike classical bits, which can only be in states of 0 or 1, qubits can exist in a superposition of states, meaning they can represent both 0 and 1 simultaneously. This property allows quantum computers to process a vast number of possibilities simultaneously.

The field emerged from quantum mechanics, which is a fundamental theory in physics that provides a description of the physical properties of nature at the atomic and subatomic scales. Quantum computing leverages quantum mechanical phenomena to perform operations on data. This can potentially solve certain problems much faster than classical computers.

For instance, Shor's algorithm demonstrates that a quantum computer can factor large integers exponentially faster than the best known algorithm running on a classical computer. This has significant implications for cryptography systems that rely on the difficulty of factoring large numbers.

---

In this example, the user anticipates the model to give explanation releavant to a 10 year old. Instead, the model produces a very technical response with an academic tone without any examples. A better response could have been (with SFT):

---

(No RLHF) *Prompt*: Explain quantum computing to a 10-year-old:

(No RLHF) *Response*: Imagine your regular computer is like playing a board game where you can only be on one space at a time. You have to move step by step to get to the end.

Quantum computers are like magical game pieces that can be in lots of different spaces at the same time! They use special bits called "qubits" instead of regular computer bits.

Regular computer bits are like light switches - they can only be ON or OFF. But qubits are special because they can be ON and OFF at the same time, kind of like how a spinning coin is neither heads nor tails until it stops.

This special power lets quantum computers solve some really tricky puzzles much faster than regular computers. Scientists are excited about them because they might help us discover new medicines, understand weather better, and solve problems that would take regular computers thousands of years!

The quantum computers we have today are still small and make mistakes, but engineers are working hard to make them bigger and better.

---


Here as you can see that the SFT model has learned to use simple language and analogies, breakdown complex concepts, avoid technical jargon like superposition and slightly adatpive to the age level of the audience

This is our goal response with SFT and RLHF :

---

(SFT RLHF) *Prompt*: Explain quantum computing to a 10-year-old:

(SFT RLHF) *Response*: Hey there! Imagine you have a super cool magic coin. When you flip a normal coin, it lands on either heads or tails, right? But this magic coin can spin and be a little bit heads and a little bit tails at the same time while it's spinning!

Quantum computers use special things called "qubits" that work like these magic coins. While normal computers use regular bits that can only be 0 OR 1 (like a coin that's only heads or tails), quantum computers use qubits that can be 0 AND 1 at the same time (like our spinning magic coin).

Why is this so awesome? Well, imagine you lost your favorite toy somewhere in your house. A normal computer would have to look in each room one by one. But a quantum computer is like having a bunch of friends who can check all the rooms at the exact same time! That makes solving certain problems MUCH faster.

Scientists are still learning how to build better quantum computers because they're tricky to make. The qubits are super sensitive - kind of like trying to keep a hundred coins spinning without letting any of them fall over!

Does that make sense? What do you think would be the coolest thing to use a quantum computer for?

---


in this example, we can clearly see the difference, it's more conversational, user friendly and uses analogies to make explanation easier like a human

## 2. Supervised Fine Tuning

<img src="assets/workflow2.png" >


### Understanding the difference between SFT and RLHF



But what if I just keep model with SFT and not RLHF? Or what if I just skip to RLHF instead of SFT?

These are excellent questions that get to the heart of why the complete RLHF pipeline exists. Supervised Fine-Tuning (SFT) and Reinforcement Learning from Human Feedback (RLHF) serve different but complementary roles in aligning language models with human expectations and preferences.

If you only implement SFT without RLHF, you'll have a model that can follow basic patterns demonstrated in your training examples, but it will struggle to generalize beyond them. As we saw in our quantum computing example, SFT can teach a model to use simpler language and appropriate analogies, but it's limited by the specific demonstrations provided. The model learns to mimic patterns without developing a deeper understanding of why certain responses are better than others. When faced with novel queries or edge cases not covered in the training data, an SFT-only model often fails to maintain the same quality of responses. Additionally, SFT can only optimize for whatever patterns exist in your demonstration data - if that data contains subtle biases or inconsistencies, those will be faithfully reproduced by the model.

Conversely, if you attempt to skip SFT and go directly to RLHF, you're likely to encounter significant challenges. RLHF works by refining an already somewhat aligned model through preference optimization. Starting with a raw pretrained model would make this process extremely inefficient and potentially unstable. The preference learning and reinforcement stages need a reasonable starting point where the model can already produce somewhat appropriate responses that humans can meaningfully compare and rank. Without SFT, the initial responses might be so far from helpful that the preference signals become too noisy or the optimization process becomes prohibitively difficult. It would be like trying to teach advanced painting techniques to someone who hasn't yet learned to hold a brush - the feedback would be overwhelming and difficult to incorporate.

The full RLHF pipeline with SFT followed by preference learning and reinforcement creates a progressively refined alignment. SFT provides the foundation by teaching the model basic response patterns and formats through demonstration. RLHF then builds on this foundation by teaching the model to distinguish between good and better responses through comparative feedback, allowing it to generalize beyond specific examples to broader human preferences. As we observed in our examples, the SFT model improved basic comprehensibility and appropriateness, while the RLHF model further enhanced engagement, conversational tone, and subtle aspects of helpfulness that are difficult to capture through demonstrations alone. This complementary relationship explains why major AI systems like ChatGPT and Claude use both techniques in sequence rather than choosing one over the other. The complete alignment process transforms raw predictive power into carefully balanced helpful assistance that respects complex human values and preferences.

### Components of SFT

To perform Supervised Fine-Tuning (SFT) on our pretrained LLM, we need high-quality demonstration data consisting of prompt-response pairs showing the desired behavior, typically thousands of examples created by experts. We also need a data preprocessing pipeline to format this data consistently, including tokenization and special tokens to distinguish between prompts and responses. SFT requires careful configuration of hyperparameters like learning rate, batch size, and optimization methods, with techniques such as warmup and decay schedules for training stability. Rather than fine-tuning all parameters, we'll use PEFT methods like LoRA that add small trainable modules while keeping most of the model frozen, making training more efficient. We'll implement a training loop for forward passes, loss calculation, backpropagation, and parameter updates, along with evaluation metrics such as perplexity and ROUGE scores to assess performance. Finally, our existing PretrainedLLM class already supports checkpointing and adapter saving, which we'll use to periodically save the model state during training. This SFT process will transform our raw model into one that can follow instructions and communicate appropriately, serving as the foundation for subsequent RLHF stages.

High-quality demonstration data: Thousands of prompt-response pairs created by experts

Data preprocessing pipeline: Consistent formatting, tokenization, and special tokens

Fine-tuning configuration: Learning rate, batch size, warmup/decay schedules

PEFT implementation: Using LoRA to add trainable modules while freezing most parameters

Training loop: Forward passes, loss calculation, backpropagation, parameter updates

Evaluation metrics: Perplexity, ROUGE scores to assess performance

Checkpointing: Saving model state during training using our existing functionality


For dataset, we're using the Databricks Dolly-15k dataset, which is a high-quality instruction-following dataset specifically designed for fine-tuning language models. This dataset contains 15,000 human-generated prompt/response pairs across various instruction categories including creative writing, classification, information extraction, open QA, brainstorming, and summarization. 

The Dolly dataset was created by Databricks employees who manually wrote both the prompts and high-quality responses, making it particularly valuable for instruction-tuning. Unlike some other datasets which may be generated or filtered from existing sources, Dolly's samples are purpose-built for teaching models to follow instructions in a helpful manner.

In [ ]:
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer, 
    TrainingArguments, 
    Trainer,
    DataCollatorForLanguageModeling
)
from peft import (
    get_peft_model,
    LoraConfig,
    TaskType,
    prepare_model_for_kbit_training
)
import os
import numpy as np
from tqdm import tqdm

class SupervisedFineTuner:
    def __init__(self, base_model, dataset_name="databricks/dolly-15k", max_seq_length=512):
        """
        Initializing SFT framework
        
        Args:
            base_model: The PretrainedLLM instance to fine-tune
            dataset_name: HuggingFace dataset identifier containing instruction/response pairs
            max_seq_length: Maximum sequence length for inputs
        """
        self.llm = base_model
        self.tokenizer = base_model.tokenizer
        self.model = base_model.model
        self.device = base_model.device
        self.max_seq_length = max_seq_length # max length of sequences that model will process
        self.dataset_name = dataset_name
        
        # If tokenizer doesn't have padding token, set it to eos token
        if self.tokenizer.pad_token is None:
            self.tokenizer.pad_token = self.tokenizer.eos_token
            self.tokenizer.pad_token_id = self.tokenizer.eos_token_id
            
        print(f"Loading dataset {dataset_name}...")
        self.raw_dataset = load_dataset(dataset_name)
        print(f"Dataset loaded with {len(self.raw_dataset['train'])} training examples")
        
    def prepare_data(self):
        """Process the dataset into the format needed for instruction fine-tuning"""
        
        def format_instruction(example):
            """Format an example into a prompt-response pair with special tokens"""
            # Different datasets have different column names, they might call different call different labels
            if 'instruction' in example and 'response' in example:
                prompt = example['instruction']
                response = example['response']
            elif 'prompt' in example and 'completion' in example:
                prompt = example['prompt']
                response = example['completion']
            else:
                # Fallback for other dataset formats
                prompt = str(example['input']) if 'input' in example else ""
                response = str(example['output']) if 'output' in example else ""
            
            # Format with special tokens
            formatted_text = f"User: {prompt.strip()}\n\nAssistant: {response.strip()}"
            return {"formatted_text": formatted_text}
        
        print("Formatting dataset...")
        self.processed_dataset = self.raw_dataset.map(format_instruction)
        
        def tokenize_function(examples):
            """Tokenize the examples and prepare for training"""
            texts = examples["formatted_text"]
            
            # Tokenize with padding and truncation
            tokenized = self.tokenizer(
                texts,
                padding="max_length",
                truncation=True,
                max_length=self.max_seq_length,
                return_tensors="pt"
            )
            
            # Create labels (for causal LM, labels are the same as input_ids)
            tokenized["labels"] = tokenized["input_ids"].clone()
            
            # Mask padding tokens in the labels to -100 so they're not included in loss
            tokenized["labels"][tokenized["input_ids"] == self.tokenizer.pad_token_id] = -100
            
            return tokenized
        
        print("Tokenizing dataset...")
        self.tokenized_dataset = self.processed_dataset.map(
            tokenize_function,
            batched=True,
            remove_columns=self.processed_dataset["train"].column_names
        )
        
        return self.tokenized_dataset
    
    def setup_peft(self, r=16, lora_alpha=32, lora_dropout=0.05):
        """Set up Parameter-Efficient Fine-Tuning using LoRA"""
        
        print("Setting up LoRA for efficient fine-tuning...")
        # Configure LoRA
        peft_config = LoraConfig(
            task_type=TaskType.CAUSAL_LM,
            r=r,  # Rank of the update matrices
            lora_alpha=lora_alpha,  # Scaling factor
            lora_dropout=lora_dropout,
            target_modules=["q_proj", "v_proj"],  # Which modules to apply LoRA to
            bias="none",
            inference_mode=False
        )
        
        # Prepare model for training
        self.model = prepare_model_for_kbit_training(self.model)
        
        # Apply LoRA
        self.model = get_peft_model(self.model, peft_config)
        
        # Display trainable parameters
        trainable_params = sum(p.numel() for p in self.model.parameters() if p.requires_grad)
        total_params = sum(p.numel() for p in self.model.parameters())
        print(f"Trainable parameters: {trainable_params:,} ({trainable_params/total_params:.2%} of total)")
        
        return self.model
    
    def train(self, output_dir="sft_model", num_epochs=3, batch_size=8, learning_rate=2e-5):
        """Train the model using the prepared dataset"""
        
        print("Setting up training arguments...")
        # Training arguments
        training_args = TrainingArguments(
            output_dir=output_dir,                           # Directory to save model checkpoints
            num_train_epochs=num_epochs,                     # Number of times to iterate through the dataset
            per_device_train_batch_size=batch_size,          # Batch size per GPU/CPU for training
            gradient_accumulation_steps=4,                   # Number of updates steps to accumulate gradients for
            warmup_ratio=0.1,                               # Percentage of steps for learning rate warmup
            weight_decay=0.01,                              # L2 regularization weight
            learning_rate=learning_rate,                     # Initial learning rate
            logging_steps=10,                               # How often to log training metrics
            save_steps=200,                                 # How often to save model checkpoints
            save_total_limit=3,                             # Maximum number of checkpoints to keep
            fp16=True if self.device == "cuda" else False,   # Whether to use 16-bit floating point precision
            report_to="none"                                # Disable external reporting services
        )
        
        # Create data collator
        data_collator = DataCollatorForLanguageModeling(
            tokenizer=self.tokenizer,
            mlm=False  # We're doing causal LM, not masked LM
        )
        
        print("Creating trainer...")
        # Initialize the trainer
        trainer = Trainer(
            model=self.model,
            args=training_args,
            train_dataset=self.tokenized_dataset["train"],
            eval_dataset=self.tokenized_dataset["test"] if "test" in self.tokenized_dataset else None,
            data_collator=data_collator
        )
        
        print("Starting training...")
        # Finally Train the model
        trainer.train()
        
        # Save the adapter
        adapter_path = os.path.join(output_dir, "adapter")
        self.model.save_pretrained(adapter_path) # save the model to the path
        self.tokenizer.save_pretrained(adapter_path)
        print(f"Saved LoRA adapter to {adapter_path}")
        
        return adapter_path
    
    def evaluate(self, evaluation_prompts):
        """Evaluate the model on a list of prompts"""
        print("Evaluating model...")
        
        for prompt in evaluation_prompts:
            print(f"Prompt: {prompt}")
            
            # Generate with the original model
            base_response = self.llm.generate(prompt, max_new_tokens=200)
            print(f"Base model response: {base_response}\n")
            
            # Format prompt for the fine-tuned model
            formatted_prompt = f"User: {prompt}\n\nAssistant: "
            inputs = self.tokenizer(formatted_prompt, return_tensors="pt").to(self.device)
            
            # Generate with the fine-tuned model
            with torch.no_grad():
                outputs = self.model.generate(
                    **inputs,
                    max_new_tokens=200,
                    temperature=0.7,
                    top_p=0.9,
                    do_sample=True
                )
                
            # Decode and display
            full_text = self.tokenizer.decode(outputs[0], skip_special_tokens=True)
            sft_response = full_text[len(formatted_prompt):]
            print(f"SFT model response: {sft_response}\n")
            print("-" * 50)



As we can see, we have first a base model (our pretrained LLM), a dataset name, and a maximum sequence length parameter that controls how much text the model processes at once. It handles tokenizer configuration by ensuring the padding token is properly set, which is crucial for consistent batch processing during training. Next, the prepare_data method loads the Dolly-15k dataset and transforms each example by extracting the prompt-response pairs and formatting them with special tokens that help the model distinguish between user input and expected output. The formatting includes adding "User:" and "Assistant:" prefixes that teach the model the proper conversation structure. After formatting, it tokenizes all examples, handling padding and truncation to ensure consistent lengths, and sets up special label handling where padding tokens are masked from loss calculations.

The setup_peft method is particularly innovative, implementing Parameter-Efficient Fine-Tuning using Low-Rank Adaptation (LoRA). Rather than updating all model weights—which would be computationally expensive—LoRA adds small trainable matrices to key attention components while keeping most parameters frozen. The method configures LoRA with appropriate rank and scaling parameters, then applies it to the query and value projection matrices of the transformer architecture. This approach dramatically reduces the number of trainable parameters to often less than 1% of the total, making fine-tuning feasible on hardware. At the core of LoRA's efficiency is its mathematical insight about weight updates. It first decomposes weight updates into products of two smaller matrices by leveraging the observation that during fine-tuning, the weight updates often have a low "intrinsic rank" - meaning they can be approximated by low-rank matrices without significant loss of information. For example, in a transformer model where a weight matrix W might be `768×768` (containing `589,824` parameters), LoRA replaces the full update with two matrices B `(768×16)` and A `(16×768)`, requiring only `24,576` parameters - a 96% reduction. These matrices are initialized with careful scaling: B starts with random Gaussian values while A begins at zero, ensuring training begins from the original model's behavior. The implementation in the code uses `r=16` for the rank hyperparameter, which determines this compression ratio. The `lora_alpha=32` parameter controls scaling during inference, effectively determining how strongly the adaptation affects the original weights. The `target_modules=["q_proj", "v_proj"]` parameter specifically targets the query and value projection matrices in the attention mechanism, which are particularly influential for language understanding and generation while leaving other components untouched.

The train method handles the actual training process by configuring optimization parameters including learning rate, batch size, and gradient accumulation steps. It sets up a training pipeline with appropriate arguments for supervised learning, including warmup schedules and weight decay for regularization. After training completes, it saves just the LoRA adapter rather than the full model, making the fine-tuned version extremely portable at just a fraction of the full model size. Finally, the evaluation method provides a convenient way to compare the base model against the fine-tuned version using the same prompts. 

In [ ]:
if __name__ == "__main__":
    
    # Initialize base model
    base_llm = PretrainedLLM(model_name="facebook/opt-350m")
    
    # Create SFT trainer
    sft = SupervisedFineTuner(base_llm, dataset_name="databricks/dolly-15k")
    
    # Prepare data
    processed_data = sft.prepare_data()
    
    # Setup PEFT
    peft_model = sft.setup_peft()
    
    # Train the model
    adapter_path = sft.train(output_dir="sft_model", num_epochs=1)  # Reduced for demo
    
    # Load the adapter into the base model
    base_llm.load_adapter(adapter_path)
    
    # Test the model
    evaluation_prompts = [
        "Explain quantum computing to a 10-year-old:",
        "Write a short story about a robot learning to feel emotions:",
        "How do I bake a chocolate cake?"
    ]
    
    sft.evaluate(evaluation_prompts)

EXAMPLE

and there we go! we have a successfully supervised fine tuned llm!

## 3. Reinforcement Learning with Human Feedback

#### Understand the relation between LLMs and Reinforcement Learning

Before diving into the human feedback mechanisms, it's crucial to understand how Reinforcement Learning fundamentals apply to language models. This connection isn't immediately obvious, as LLMs seem quite different from traditional RL scenarios like game-playing agents or robotic control systems.


Reinforcement Learning is a machine learning paradigm where an **agent** learns to make optimal decisions by interacting with an **environment** to maximize cumulative rewards over time. The mathematical foundation rests on the Markov Decision Process (MDP), formally defined as a tuple $(S, A, P, R, \gamma)$ where:

**Core RL Components:**
- **State Space (S)**: The set of all possible situations the agent can observe
- **Action Space (A)**: The set of all possible decisions the agent can make  
- **Transition Probabilities (P)**: $P(s'|s,a)$ - probability of reaching state $s'$ from state $s$ taking action $a$
- **Reward Function (R)**: $R(s,a,s')$ - immediate feedback signal for taking action $a$ in state $s$
- **Policy (π)**: $\pi(a|s)$ - strategy mapping states to action probabilities
- **Value Function (V)**: $V^\pi(s) = \mathbb{E}_\pi[\sum_{t=0}^{\infty} \gamma^t R_{t+1} | S_0 = s]$ - expected cumulative reward
- **Discount Factor (γ)**: Controls the importance of future vs. immediate rewards

Let's return to our quantum computing explanation scenario to see how these RL concepts map to language models. Imagine our model is tasked with explaining quantum computing to a 10-year-old:

**Traditional RL Scenario**: A robot learning to navigate a maze
- **Environment**: Physical maze with walls and pathways
- **State**: Robot's current position and orientation
- **Actions**: Move forward, turn left, turn right, etc.
- **Reward**: +10 for reaching the goal, -1 for hitting walls
- **Policy**: Strategy for choosing movements based on current position

**LLM RL Scenario**: Model explaining quantum computing to a child
- **Environment**: The conversational context and task requirements
- **State**: Current prompt + all previously generated tokens
- **Actions**: Choosing the next token from vocabulary (~50,000 possibilities)
- **Reward**: Human preference scores for the complete explanation
- **Policy**: Strategy for choosing tokens to maximize helpfulness

At each generation step, the model is in a specific state (the prompt plus all previously generated tokens) and must choose an action (the next token). For example:

- **State 1**: "Explain quantum computing to a 10-year-old:"
- **Action 1**: Choose token "Hey" (starting conversationally)
- **State 2**: "Explain quantum computing to a 10-year-old: Hey"
- **Action 2**: Choose token "there!" (continuing the friendly tone)
- **State 3**: "Explain quantum computing to a 10-year-old: Hey there!"
- **Action 3**: Choose token "Imagine" (starting an analogy)

And so forth, until the model generates an end-of-sequence token, completing the trajectory.

Let's formalize this mapping with concrete mathematical examples:

**State Representation in LLMs:**
In traditional RL, a state might be discrete coordinates: $s = (x, y, \theta)$
In LLMs, the state is the sequence of tokens generated so far:
$$s_t = (w_1, w_2, \ldots, w_t)$$

For our quantum example:
- $s_0$: "Explain quantum computing to a 10-year-old:"
- $s_1$: "Explain quantum computing to a 10-year-old: Hey"  
- $s_2$: "Explain quantum computing to a 10-year-old: Hey there!"
- $s_3$: "Explain quantum computing to a 10-year-old: Hey there! Imagine"

**Action Space in LLMs:**
The action space is the model's vocabulary $\mathcal{V}$. At each step $t$, the model chooses action $a_t \in \mathcal{V}$:
$$a_t = \text{NextToken}(s_t)$$

The probability of choosing action $a$ in state $s$ is given by the softmax over logits:
$$\pi_\theta(a|s) = \frac{\exp(f_\theta(s,a))}{\sum_{a' \in \mathcal{V}} \exp(f_\theta(s,a'))}$$
where $f_\theta(s,a)$ represents the logit score for token $a$ given context $s$.

**Logit Scores**: These are the raw, unnormalized output values from the neural network before applying softmax. The logit score indicates how strongly the model believes a particular token should follow the current context - higher logits mean higher probability after softmax normalization. These scores directly impact which tokens the model considers most likely to continue the sequence.

**Policy Evolution:**
The model's policy starts as next-token prediction trained on internet text:
$$\pi_{\text{pretrained}}(a_t|s_t) \propto P_{\text{internet}}(a_t|s_t)$$

Through RLHF, this evolves to optimize for human preferences:
$$\pi_{\text{RLHF}}(a_t|s_t) = \arg\max_\pi \mathbb{E}_{s,a \sim \pi}[R_{\text{human}}(s,a)]$$

Where:
- $\pi_{\text{RLHF}}$ is the optimized policy after reinforcement learning
- $a_t$ is the next token to generate
- $s_t$ is the current context (all tokens so far)
- $\arg\max_\pi$ means "find the policy that maximizes"
- $\mathbb{E}_{s,a \sim \pi}$ is the expected value when sampling states and actions from policy $\pi$
- $R_{\text{human}}(s,a)$ is the human preference reward for taking action $a$ in state $s$


A **trajectory** $\tau$ in RL is a sequence of state-action-reward tuples:
$$\tau = (s_0, a_0, r_0, s_1, a_1, r_1, \ldots, s_T, a_T, r_T)$$

In our quantum explanation scenario:
- $s_0$: "Explain quantum computing to a 10-year-old:"
- $a_0$: "Hey" (choosing conversational start)
- $s_1$: "Explain quantum computing to a 10-year-old: Hey"
- $a_1$: "there!" (continuing friendly tone)
- $s_2$: "Explain quantum computing to a 10-year-old: Hey there!"
- $a_2$: "Imagine" (starting an analogy)
- ...
- $r_T$: Human preference score for complete response

**The Sparse Reward Challenge:**
Unlike traditional RL where rewards can be immediate (hitting a wall = -1), language model rewards are typically **sparse** - only available at the end of generation when humans evaluate the complete response. This creates the credit assignment problem: which early token choices led to the eventual positive feedback?

$$R(s_t, a_t) = \begin{cases} 
0 & \text{if } t < T \\
R_{\text{human}}(\text{full response}) & \text{if } t = T
\end{cases}$$

The genius of applying RL to language models lies in reframing text generation as a sequential decision-making process. Instead of just predicting the statistically most likely next token (as in standard pretraining), we can now optimize for much more sophisticated objectives:

**Traditional Approach**: 
$$\text{Objective: } \max_\theta \mathbb{E}_{(x,y) \sim D}[\log P_\theta(y|x)]$$
"What token typically comes next in internet text?"

where:
- $\theta$ represents the model parameters being optimized
- $\mathbb{E}_{(x,y) \sim D}$ is the expected value over examples from dataset D
- $(x,y)$ are input-output pairs (prompt and completion)
- $P_\theta(y|x)$ is the probability the model assigns to output y given input x
- The objective maximizes log probability of correct completions in the training data

**RL Approach**:
$$\text{Objective: } \max_\theta \mathbb{E}_{\tau \sim \pi_\theta}[R(\tau)]$$
"What token choice will lead to responses humans find helpful, harmless, and honest?"

where:
- $\theta$ represents the model parameters being optimized
- $\mathbb{E}_{\tau \sim \pi_\theta}$ is the expected value over trajectories sampled from policy $\pi_\theta$
- $\tau$ is a complete trajectory (sequence of tokens) generated by the model
- $R(\tau)$ is the reward (human preference score) assigned to the complete response
- The objective maximizes the expected reward across all possible response trajectories

In our quantum explanation example, the traditional model might continue with "superposition" and "entanglement" because they frequently appear after "quantum computing" in training data. But an RL-optimized model learns that choosing words like "imagine" or "think of it like" leads to better human ratings for child-appropriate explanations.

**State Space Complexity**: Unlike board games where states are well-defined, language model states live in a vast combinatorial space. The same prompt can lead to exponentially many possible conversation paths:

- Path 1: "Hey there! Imagine you have a magic coin..."
- Path 2: "So, quantum computers are like super special computers..."  
- Path 3: "You know how regular computers work with..."

Each path represents a different trajectory through the state space, leading to different human preference scores.

This is the exact reason why having same prompt to LLM most likely won't give you the same response 

**The Value Function in Language Models:**
The value function estimates expected future rewards from any given state:
$$V^\pi(s) = \mathbb{E}_{\pi}[\sum_{t'=t}^T \gamma^{t'-t} R_{t'}|S_t = s]$$

where:
- $V^\pi(s)$: Expected total reward when following policy $\pi$ from state $s$ (a particular sequence of tokens)
- $\mathbb{E}_{\pi}$: Expected value when following token selection policy $\pi$
- $\sum_{t'=t}^T$: Sum of rewards from current time $t$ to end of generation $T$
- $\gamma^{t'-t}$: Discount factor giving less weight to distant rewards
- $R_{t'}$: Reward received at time $t'$ (usually 0 until final human feedback)
- $S_t = s$: Starting from the current state (current prompt + generated tokens so far)

In our quantum explanation context:
- High value states: Contexts that lead to engaging, age-appropriate explanations
- Low value states: Contexts that lead to technical jargon or confusion

For example:
- $V("Explain quantum computing: Hey there! Imagine")$ = High (leads to good analogies)
- $V("Explain quantum computing: Quantum superposition involves")$ = Low (too technical)


This reframing transforms language model training from pattern matching to goal-oriented behavior, enabling models to optimize for nuanced human values rather than just statistical likelihood. The mathematical framework provides:

1. **Credit Assignment**: Understanding which early tokens contribute to eventual success
2. **Policy Optimization**: Systematic improvement toward human preferences  
3. **Value Estimation**: Predicting which conversation paths lead to better outcomes
4. **Exploration-Exploitation**: Balancing trying new approaches vs. using known good patterns

The subsequent RLHF stages (preference collection, reward modeling, and policy optimization) all build on this fundamental RL foundation to create AI systems that are not just knowledgeable, but genuinely helpful and aligned with human intentions. This mathematical rigor is what enables the sophisticated preference learning and optimization algorithms we'll explore in the following sections.

### Human Preference Collection

#### But Why Do We Even Need Human Preference Collection?

You might wonder: "We already have an SFT model that can explain quantum computing nicely to children. Why do we need this additional step?" This is a fundamental question that gets to the heart of what makes RLHF so powerful.

The limitation of SFT becomes clear when we consider what it actually teaches the model. SFT is essentially sophisticated pattern matching - it learns to mimic the style and structure of the demonstration data, but it doesn't develop a deeper understanding of *why* certain responses are better than others. It's like teaching someone to paint by having them copy masterpieces stroke by stroke - they might reproduce the paintings accurately, but they haven't learned the principles that make great art.

Consider our quantum computing example. An SFT model might learn that responses for children should use simple language and analogies, but it lacks nuanced understanding of what makes one analogy better than another. It might consistently use the "spinning coin" analogy because it appeared in training data, but it won't know that sometimes a "magic treasure hunt" analogy might be more engaging for a particular child, or that asking follow-up questions creates better learning experiences.

Human Preference Collection addresses these limitations by teaching the model to understand *relative quality* - not just how to generate appropriate responses, but how to distinguish between good, better, and best responses. This comparative learning is fundamentally different from demonstration learning and unlocks much more sophisticated behavior.


<img src="assets/workflow3.png" >


Human Preference Collection is essentially a massive, systematic comparison exercise where human evaluators help the model learn what "better" means in countless different scenarios. Here's how it works in practice:

#### Step 1: Response Generation
First, we use our SFT model to generate multiple responses to the same prompt. For our quantum computing example, we might generate several different explanations:

**Prompt**: "Explain quantum computing to a 10-year-old"

**Response A**: "Quantum computers use qubits instead of regular bits. Think of regular bits like light switches that can only be on or off. Qubits are special because they can be on and off at the same time, like a spinning coin that's both heads and tails until it stops spinning."

**Response B**: "Hey there! Imagine you have a magic coin that can be heads and tails at the same time while it's spinning! Quantum computers use these special 'qubits' that work just like our magic coin. This lets them solve puzzles much faster than regular computers. What's your favorite kind of puzzle?"

**Response C**: "Quantum computing is like having a super-fast helper that can check every room in your house for your lost toy at the same time, instead of checking one room after another like a regular computer would do."

#### Step 2: Human Evaluation and the Mathematics of Preference

Human evaluators (often experts in education, communication, or the relevant domain) are presented with pairs of these responses and asked to choose which one better fulfills the criteria. The evaluation interface might look like:

```
Prompt: Explain quantum computing to a 10-year-old

Response A: [Response A text]
Response B: [Response B text]

Which response better explains quantum computing to a 10-year-old?
□ Response A is significantly better
□ Response A is slightly better  
□ Response B is slightly better
□ Response B is significantly better
□ They're about the same quality
```

The evaluators consider multiple factors:
- **Age-appropriateness**: Does it use language a 10-year-old would understand?
- **Engagement**: Would this capture and maintain a child's interest?
- **Accuracy**: Is the explanation scientifically sound (even if simplified)?
- **Clarity**: Would a child actually understand this after reading it?
- **Interactivity**: Does it encourage questions or further learning?

#### The Bradley-Terry Model: Mathematical Foundation of Preference

Behind this seemingly simple comparison process lies sophisticated mathematical modeling. The **Bradley-Terry model** provides the theoretical foundation for how we convert human preferences into trainable signals. This model, originally developed for analyzing sports competitions, perfectly captures the essence of pairwise comparisons.

**The Core Mathematical Insight:**
The Bradley-Terry model assumes that each response has an underlying "quality score" or "strength" that determines how likely it is to be preferred over another response. If response $y_w$ (the "winner") has strength $\pi_w$ and response $y_l$ (the "loser") has strength $\pi_l$, then the probability that humans prefer $y_w$ over $y_l$ is:

$$P(y_w \succ y_l) = \frac{\pi_w}{\pi_w + \pi_l}$$

In our quantum computing example, if Response B consistently beats Response A in human evaluations, the model learns that Response B has higher intrinsic quality for this type of explanation.

**Connecting to Language Models:**
In RLHF, we don't observe these strength values $\pi$ directly. Instead, we model them using our language model's probability of generating each response:

$$P(y_w \succ y_l | x) = \frac{\exp(r_\theta(x, y_w))}{\exp(r_\theta(x, y_w)) + \exp(r_\theta(x, y_l))}$$

where:
- $x$ is the prompt ("Explain quantum computing to a 10-year-old")
- $y_w$ and $y_l$ are the competing responses
- $r_\theta(x, y)$ is a reward function that scores how good response $y$ is for prompt $x$

This can be simplified using the logistic function:
$$P(y_w \succ y_l | x) = \sigma(r_\theta(x, y_w) - r_\theta(x, y_l))$$

where $\sigma$ is the sigmoid function: $\sigma(z) = \frac{1}{1 + e^{-z}}$, which transforms any input into a value between 0 and 1, essentially converting the difference in reward scores into a probability.

**Why This Mathematical Foundation Matters:**
The Bradley-Terry model provides several crucial advantages:

1. **Transitivity**: If Response B beats Response A, and Response C beats Response B, then Response C should beat Response A. The model ensures these relationships are mathematically consistent.

2. **Calibrated Probabilities**: The model provides well-calibrated probability estimates. If the model says Response B has a 0.8 probability of being preferred over Response A, then in practice, humans should prefer B about 80% of the time.

3. **Handling Uncertainty**: The model can represent cases where responses are very close in quality (probability near 0.5) versus clear winners (probability near 0 or 1).

#### Step 3: Preference Data Creation and Loss Functions

Each comparison creates a preference data point. If evaluators consistently prefer Response B over Response A, this creates a training signal that Response B's approach (conversational tone, asking questions, using engaging analogies) should be valued more highly than Response A's approach (more technical, less interactive).

**The Preference Dataset:**
Mathematically, our preference dataset $\mathcal{D}$ consists of tuples:
$$\mathcal{D} = \{(x^{(i)}, y_w^{(i)}, y_l^{(i)})\}_{i=1}^N$$

where for each prompt $x^{(i)}$, we have identified a preferred response $y_w^{(i)}$ and a less preferred response $y_l^{(i)}$.

**The Loss Function:**
To train models on this preference data, we use the Bradley-Terry loss:
$$\mathcal{L}_{BT} = -\sum_{i=1}^N \log \sigma(r_\theta(x^{(i)}, y_w^{(i)}) - r_\theta(x^{(i)}, y_l^{(i)}))$$

This loss function represents the Bradley-Terry loss for preference learning, where:
- $\mathcal{L}_{BT}$ is the total loss we're minimizing
- $\sigma$ is the sigmoid function that converts score differences into probabilities
- $r_\theta(x^{(i)}, y_w^{(i)})$ is the reward score for the preferred response
- $r_\theta(x^{(i)}, y_l^{(i)})$ is the reward score for the less preferred response
- The negative log ensures that maximizing the probability of correct preferences minimizes the loss
- We sum over all N preference pairs in our dataset

This loss function encourages the reward model to assign higher scores to preferred responses and lower scores to dispreferred ones.

**Practical Example with Our Quantum Responses:**
Let's say human evaluators prefer Response B over Response A with a probability of 0.85. The Bradley-Terry model would encode this as:

$$P(\text{Response B} \succ \text{Response A}) = \sigma(r_\theta(\text{prompt}, \text{Response B}) - r_\theta(\text{prompt}, \text{Response A})) = 0.85$$

This means:
$$r_\theta(\text{prompt}, \text{Response B}) - r_\theta(\text{prompt}, \text{Response A}) = \sigma^{-1}(0.85) \approx 1.73$$

The reward model learns that Response B's engaging, interactive approach should score about 1.73 points higher than Response A's more technical approach.


Let's extend our quantum computing analogy to understand preference collection itself:

Imagine you're teaching a young student not just about quantum computers, but about how to *mathematically evaluate* different explanations. Instead of just showing them one "correct" way to explain it (like SFT), you show them pairs of explanations and ask: "Which of these would help you understand better, and how confident are you?"

**Comparison 1**: Technical explanation vs. Magic coin analogy
→ Student strongly prefers: Magic coin analogy (Bradley-Terry probability: 0.9)

**Comparison 2**: Magic coin analogy vs. Treasure hunt analogy  
→ Student slightly prefers: Treasure hunt analogy (Bradley-Terry probability: 0.6)

**Comparison 3**: Treasure hunt analogy vs. Interactive treasure hunt with questions
→ Student strongly prefers: Interactive version (Bradley-Terry probability: 0.95)

The mathematical model learns not just the ranking (interactive > treasure hunt > magic coin > technical), but also the *strength* of these preferences. The large gap between technical and magic coin explanations tells the model that engagement is critically important. The smaller gap between magic coin and treasure hunt analogies suggests that while both work well, the specific analogy choice is less crucial than the overall approach.


In real RLHF implementations, this process happens at massive scale:

- **Thousands of prompts** across diverse topics and scenarios
- **Multiple responses** generated for each prompt (typically 2-4)
- **Multiple evaluators** rating each comparison (to ensure reliability)
- **Hundreds of thousands** of preference comparisons collected
- **Quality control** measures to ensure consistent evaluation standards

**Inter-Annotator Agreement:**
With multiple human evaluators, we need to measure consistency. If evaluators frequently disagree, the preference signal becomes noisy. We typically use metrics like:

- **Fleiss' Kappa**: Measures agreement across multiple evaluators
- **Krippendorff's Alpha**: Handles different scales of preference (slight vs. strong)

**Confidence Intervals:**
The Bradley-Terry model also provides uncertainty estimates. If only 55% of evaluators prefer Response B over Response A, the confidence interval for the preference probability might be [0.45, 0.65], indicating high uncertainty. The training process can weight these uncertain comparisons less heavily.

For our quantum explanation example, the preference data might reveal patterns like:
- Responses with questions score higher than statements (BT probability: 0.75)
- Analogies to familiar objects beat abstract concepts (BT probability: 0.88)
- Conversational tone ("Hey there!") beats formal tone (BT probability: 0.82)
- Interactive explanations beat passive ones (BT probability: 0.91)

These probabilities encode not just the preferences, but their relative importance for training the reward model.


**Handling Ties and Ambiguous Preferences:**
Sometimes evaluators can't decide between responses, or they're genuinely equivalent. The Bradley-Terry model handles this by allowing preference probabilities near 0.5. We can extend the model to explicitly handle ties:

$$P(\text{tie}) = \frac{\nu}{\pi_w + \pi_l + \nu}$$

where $\nu$ represents the "tie strength."

**Multiple Response Comparisons:**
While pairwise comparisons are most common, we can extend to ranking multiple responses simultaneously using the **Plackett-Luce model**, which generalizes Bradley-Terry to full rankings:

$$P(\text{ranking: } y_1 \succ y_2 \succ \ldots \succ y_k) = \prod_{i=1}^k \frac{\pi_{y_i}}{\sum_{j=i}^k \pi_{y_j}}$$

This equation represents the Plackett-Luce model for comparing multiple responses simultaneously. Here:
- $y_1 \succ y_2 \succ \ldots \succ y_k$ is a complete ranking of k responses
- $\pi_{y_i}$ is the quality/strength score of response $y_i$
- The equation calculates the probability of observing a specific ranking by multiplying the conditional probabilities of each response being selected as best among the remaining options
- At each step i, we calculate the probability of choosing response $y_i$ as the best among all remaining responses ($y_i$ through $y_k$)

**Temporal and Context Dependencies:**
Advanced implementations consider that preferences might change over time or depend on context. The model can be extended to:

$$P(y_w \succ y_l | x, t, c) = \sigma(r_\theta(x, y_w, t, c) - r_\theta(x, y_l, t, c))$$

This extended equation models preferences that change over time or depend on context. Here:
- $t$ represents the time component (preferences might evolve as language norms change)
- $c$ represents additional context factors (user demographics, device type, previous interactions)
- $r_\theta(x, y, t, c)$ is the reward function that scores response $y$ for prompt $x$, considering time $t$ and context $c$
- $\sigma$ is the sigmoid function that converts the difference in reward scores into a probability
- The equation gives the probability that response $y_w$ is preferred over $y_l$ given all these factors

This enables more personalized and contextually appropriate preference modeling beyond static comparisons.

where $t$ represents time and $c$ represents additional context.

#### Why This Works Better Than More SFT Data

You might think: "Why not just collect more demonstration data instead?" The key insight is that preference collection captures information that demonstration data cannot:

**Demonstration data tells us**: "This is a good response" (provides one point in quality space)

**Preference data tells us**: "This response is better than that response *because*..." (provides relative quality differences)

**Mathematical Perspective:**
SFT learns to maximize likelihood:
$$\mathcal{L}_{SFT} = \sum_{i=1}^N \log P_\theta(y^{(i)} | x^{(i)})$$

Where:
- $\mathcal{L}_{SFT}$ is the supervised fine-tuning loss function we're maximizing
- $P_\theta(y^{(i)} | x^{(i)})$ is the probability that our model with parameters $\theta$ assigns to the demonstration response $y^{(i)}$ given prompt $x^{(i)}$
- We sum over all N examples in our demonstration dataset
- This function encourages the model to assign high probability to human-written demonstration responses, but doesn't distinguish between different quality levels among acceptable responses

This doesn't distinguish between "good" and "better" - it treats all demonstration responses as equally optimal.

Preference learning optimizes for relative quality:
$$\mathcal{L}_{Pref} = \sum_{i=1}^N \log P(y_w^{(i)} \succ y_l^{(i)} | x^{(i)})$$

This explicitly learns the quality differences that matter to humans. In this equation:
- $\mathcal{L}_{Pref}$ is the preference learning loss function we're maximizing
- $P(y_w^{(i)} \succ y_l^{(i)} | x^{(i)})$ is the probability that the preferred response $y_w^{(i)}$ is indeed better than the less preferred response $y_l^{(i)}$ for prompt $x^{(i)}$
- We sum over all N preference pairs in our dataset
- This function encourages the model to correctly predict human preferences between responses, learning the relative quality differences humans care about

The "because" part is crucial. Through comparative evaluation, the model learns the underlying principles that make responses effective, not just specific examples of effective responses. This enables much better generalization to new scenarios and more nuanced quality judgments.

In our quantum computing scenario, instead of learning "use the magic coin analogy," the model learns "use analogies that relate to a child's everyday experience, and frame them in an engaging, interactive way" with a quantified strength of preference (Bradley-Terry probability of 0.83 for interactive vs. passive explanations).

This preference data becomes the foundation for the next stage: training a reward model that can automatically evaluate response quality and guide the final optimization process. The human preferences, collected at scale and mathematically modeled through the Bradley-Terry framework, teach the AI system not just what to say, but what makes some ways of saying things better than others - and exactly how much better.

### Reward Model Training



<img src="assets/workflow4.png" >



Now that we have collected thousands of human preference comparisons, we face a critical challenge: how do we scale this human judgment to evaluate millions of potential responses? This is where reward models become essential - they serve as computational proxies for human preferences, transforming our carefully collected preference data into an automated evaluation system.

#### Why Do We Need Reward Models?

You might wonder: "Why can't we just ask humans to evaluate every response during training?" The scale problem becomes apparent when we consider the numbers:

- **During preference collection**: Humans evaluate ~100,000 preference pairs over weeks or months
- **During RL optimization**: The model generates millions of responses during training, each needing evaluation
- **Real-time constraints**: Policy optimization requires immediate feedback for each generated token sequence

Having humans evaluate every single response would be impossibly slow and expensive. We need a way to "distill" human judgment into a fast, automated system that can predict human preferences at scale.


Remember our Bradley-Terry model from preference collection? The reward model is where that mathematical framework becomes operational. We're essentially training a neural network to predict the outcome of human preference comparisons.

**The Core Insight:**
If humans prefer response $y_w$ over $y_l$ for prompt $x$, then there exists some underlying "reward" or "quality score" such that:

$$r(x, y_w) > r(x, y_l)$$

The Bradley-Terry model tells us the probability of this preference should be:
$$P(y_w \succ y_l | x) = \sigma(r(x, y_w) - r(x, y_l))$$

**Training Objective:**
Our reward model $r_\theta$ is trained to minimize the Bradley-Terry loss:
$$\mathcal{L}_{RM} = -\sum_{i=1}^N \log \sigma(r_\theta(x^{(i)}, y_w^{(i)}) - r_\theta(x^{(i)}, y_l^{(i)}))$$

This loss function encourages the reward model to assign higher scores to preferred responses and lower scores to dispreferred ones.


Let's continue with our educational tutor scenario. Imagine we've collected preference data and found these patterns:

**Preference Pattern 1**: Interactive explanations consistently beat passive ones (85% preference rate)
**Preference Pattern 2**: Age-appropriate analogies beat technical terms (92% preference rate)  
**Preference Pattern 3**: Encouraging tone beats neutral tone (73% preference rate)

Instead of memorizing these specific comparisons, we want to train a "preference predictor" (our reward model) that can:

1. **Generalize**: Evaluate new quantum explanations it has never seen before
2. **Combine factors**: Understand that an explanation that is both interactive AND uses good analogies should score higher than one with just good analogies
3. **Quantify quality**: Assign numerical scores that reflect the strength of human preferences

For our quantum computing examples:

- **Response A** (technical): "Quantum computers use qubits that exist in superposition..."
  - Reward score: 2.1 (low due to technical language)

- **Response B** (good analogy): "Think of qubits like spinning coins that can be heads and tails..."
  - Reward score: 6.8 (higher due to good analogy)

- **Response C** (interactive + analogy): "Hey there! Imagine you have a magic coin... What do you think would happen if...?"
  - Reward score: 8.9 (highest due to combining engagement, analogy, and interactivity)

#### Architecture 

**Model Architecture:**
The reward model typically shares the same transformer architecture as our language model, but with a crucial difference in the output layer:

```
Input: [Prompt + Response tokens]
     ↓
Transformer Layers (shared with LLM)
     ↓
Final Hidden State
     ↓
Linear Projection → Single Scalar Score
```

**Key Architectural Decisions:**

1. **Shared Backbone**: Using the same transformer architecture ensures the reward model understands language with the same sophistication as the policy model

2. **Scalar Output**: Unlike language models that output probability distributions over vocabulary, reward models output a single number representing quality

3. **Prompt-Response Processing**: The model sees the entire conversation context (prompt + response) and learns to evaluate the appropriateness of the response given the specific prompt


**Dataset Preparation:**
Our preference dataset becomes a set of training examples:
$$\mathcal{D}_{RM} = \{(x^{(i)}, y_w^{(i)}, y_l^{(i)})\}_{i=1}^N$$

Each example consists of:
- $x^{(i)}$: A prompt ("Explain quantum computing to a 10-year-old")
- $y_w^{(i)}$: The preferred response (interactive explanation with analogies)
- $y_l^{(i)}$: The less preferred response (technical explanation)

**Forward Pass:**
For each training example, we compute:

$r_w = r_\theta(x^{(i)}, y_w^{(i)})$ (reward for preferred response)

$r_l = r_\theta(x^{(i)}, y_l^{(i)})$ (reward for less preferred response)

In this forward pass, we're computing reward scores for both responses in a preference pair. The reward model $r_\theta$ with parameters $\theta$ processes each prompt-response pair independently. It assigns a scalar score $r_w$ to the preferred response $y_w^{(i)}$ and a score $r_l$ to the less preferred response $y_l^{(i)}$, both conditioned on the same prompt $x^{(i)}$. This forward pass transforms text into quantitative preference signals that can be used to calculate the loss and update model parameters.

**Loss Calculation:**
The Bradley-Terry loss becomes:

$$\mathcal{L} = -\log \sigma(r_w - r_l) = -\log \frac{1}{1 + e^{-(r_w - r_l)}}$$

This can be rewritten as:
$$\mathcal{L} = \log(1 + e^{-(r_w - r_l)}) = \log(1 + e^{r_l - r_w})$$

The Bradley-Terry loss function mathematically quantifies how well our reward model predicts human preferences. This equation represents the negative log likelihood of the probability that response $y_w$ is preferred over response $y_l$. It penalizes the model when it assigns similar or incorrect scores to the preference pairs. When the preferred response has a much higher reward score than the less preferred one ($r_w \gg r_l$), the loss approaches zero. Conversely, if the model incorrectly scores the less preferred response higher, the loss grows substantially. This elegant formulation ensures the reward model learns to align its numerical scores with human judgments across diverse responses.

**Gradient Flow:**
The gradients encourage:
- **Increasing $r_w$**: Making preferred responses score higher
- **Decreasing $r_l$**: Making dispreferred responses score lower
- **Maximizing the gap**: The sigmoid function creates stronger gradients when the difference is small, encouraging clear separation


**Preference Strength Modeling:**
Not all preferences are equally strong. We can extend the model to handle preference strength:

$$P(y_w \succ y_l | x, s) = \sigma(s \cdot (r_\theta(x, y_w) - r_\theta(x, y_l)))$$

where $s$ represents preference strength:
- $s = 1$: Slight preference  
- $s = 2$: Strong preference
- $s = 3$: Very strong preference

**Regularization and Calibration:**
To prevent overfitting and ensure well-calibrated probabilities, we often add regularization terms:

$$\mathcal{L}_{total} = \mathcal{L}_{BT} + \lambda_1 \|\theta\|_2^2 + \lambda_2 \mathcal{L}_{calibration}$$

where:
- $\|\theta\|_2^2$ is L2 regularization preventing large weights
- $\mathcal{L}_{calibration}$ ensures predicted probabilities match observed frequencies

The equation represents the total loss function used to train the reward model, combining the Bradley-Terry preference loss with regularization terms. L2 regularization (the $\|\theta\|_2^2$ term) penalizes large parameter values by adding the sum of squared weights to the loss function, encouraging the model to learn simpler patterns and avoid overfitting. This regularization technique shrinks weights toward zero, improving generalization to unseen data by preventing the model from becoming too complex or putting too much emphasis on any single feature.


**Handling Uncertainty:**

For ambiguous preferences where humans disagreed, we can model uncertainty:

$$\mathcal{L}_{uncertain} = -\alpha \log \sigma(r_w - r_l) - (1-\alpha) \log \sigma(r_l - r_w)$$

where $\alpha$ represents the fraction of annotators who preferred $y_w$.

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset
from tqdm import tqdm
import os

class RewardModel(nn.Module):
    def __init__(self, base_model, device="cuda"):
        """
        Initialize reward model using our existing PretrainedLLM class
        
        Args:
            base_model: PretrainedLLM instance (our existing class)
            device: Computing device
        """
        super().__init__()
        self.device = device
        
        # Use the transformer from our existing PretrainedLLM class
        self.transformer = base_model.model
        self.tokenizer = base_model.tokenizer
        
        # Remove the language modeling head and add reward head
        if hasattr(self.transformer, 'lm_head'):
            self.transformer.lm_head = None
        
        # Single scalar output for reward prediction
        hidden_size = self.transformer.config.hidden_size
        self.reward_head = nn.Linear(hidden_size, 1)
        
        # Initialize reward head with small weights
        nn.init.normal_(self.reward_head.weight, std=0.01)
        nn.init.zeros_(self.reward_head.bias)
        
        print(f"Reward model initialized with {sum(p.numel() for p in self.parameters())/1e6:.1f}M parameters")
        
    def forward(self, input_ids, attention_mask=None):
        """
        Forward pass through the reward model
        
        Args:
            input_ids: Token IDs for [prompt + response]
            attention_mask: Mask for padding tokens
            
        Returns:
            reward_score: Scalar reward for the input sequence
        """
        # Get transformer outputs (compatible with our transformer architecture)
        outputs = self.transformer(
            input_ids=input_ids,
            attention_mask=attention_mask,
            output_hidden_states=True
        )
        
        # Get the last hidden state
        last_hidden_state = outputs.hidden_states[-1]
        
        # Use the last non-padding token for reward prediction
        if attention_mask is not None:
            # Find the last non-padding token for each sequence
            last_non_pad_indices = attention_mask.sum(dim=1) - 1
            batch_size = input_ids.shape[0]
            
            # Extract the hidden state at the last non-padding position
            last_hidden = last_hidden_state[
                torch.arange(batch_size), last_non_pad_indices
            ]
        else:
            # If no mask, use the last token
            last_hidden = last_hidden_state[:, -1, :]
        
        # Project to scalar reward
        reward_score = self.reward_head(last_hidden).squeeze(-1)
        
        return reward_score

class RewardModelTrainer:
    def __init__(self, reward_model, learning_rate=1e-5):
        """
        Initialize reward model trainer
        
        Args:
            reward_model: RewardModel instance
            learning_rate: Learning rate for optimization
        """
        self.model = reward_model
        self.tokenizer = reward_model.tokenizer
        self.device = reward_model.device
        self.optimizer = torch.optim.AdamW(reward_model.parameters(), lr=learning_rate)
        
    def create_preference_dataset(self, sft_model, prompts, num_responses_per_prompt=4):
        """
        Create synthetic preference dataset using our SFT model
        This simulates the human preference collection process
        
        Args:
            sft_model: SupervisedFineTuner instance (our SFT model)
            prompts: List of prompts to generate responses for
            num_responses_per_prompt: Number of responses to generate per prompt
            
        Returns:
            preference_data: List of preference tuples (prompt, preferred, dispreferred)
        """
        print("Generating responses for preference dataset...")
        preference_data = []
        
        for prompt in tqdm(prompts, desc="Generating preference data"):
            responses = []
            
            # Generate multiple responses for the same prompt
            for _ in range(num_responses_per_prompt):
                # Format prompt for SFT model
                formatted_prompt = f"User: {prompt}\n\nAssistant: "
                inputs = self.tokenizer(formatted_prompt, return_tensors="pt").to(self.device)
                
                with torch.no_grad():
                    outputs = sft_model.model.generate(
                        **inputs,
                        max_new_tokens=150,
                        temperature=0.8,  # Higher temperature for diversity
                        top_p=0.9,
                        do_sample=True,
                        pad_token_id=self.tokenizer.eos_token_id
                    )
                
                # Decode response
                full_text = self.tokenizer.decode(outputs[0], skip_special_tokens=True)
                response = full_text[len(formatted_prompt):]
                responses.append(response.strip())
            
            # Create preference pairs using simple heuristics
            # In practice, this would be done by human evaluators
            for i in range(len(responses)):
                for j in range(i + 1, len(responses)):
                    # Simple heuristic: prefer longer, more engaging responses
                    # In reality, humans would judge based on helpfulness, accuracy, etc.
                    score_i = self._simple_quality_score(responses[i])
                    score_j = self._simple_quality_score(responses[j])
                    
                    if score_i > score_j:
                        preference_data.append((prompt, responses[i], responses[j]))
                    elif score_j > score_i:
                        preference_data.append((prompt, responses[j], responses[i]))
        
        print(f"Created {len(preference_data)} preference pairs")
        return preference_data
    
    def _simple_quality_score(self, response):
        """
        Simple heuristic to simulate human preference
        In practice, this would be actual human evaluation
        """
        score = 0
        
        # Prefer responses with questions (interactive)
        if '?' in response:
            score += 2
        
        # Prefer responses with engaging words
        engaging_words = ['imagine', 'think', 'consider', 'hey', 'wow', 'amazing']
        for word in engaging_words:
            if word.lower() in response.lower():
                score += 1
        
        # Prefer moderate length (not too short, not too long)
        length = len(response.split())
        if 20 <= length <= 100:
            score += 1
        elif length < 10:
            score -= 2
        
        # Prefer responses with analogies
        analogy_words = ['like', 'similar', 'imagine', 'think of']
        for word in analogy_words:
            if word.lower() in response.lower():
                score += 1
        
        return score
    
    def prepare_preference_batch(self, preference_data, batch_size=8):
        """
        Prepare batches for training the reward model
        
        Args:
            preference_data: List of (prompt, preferred, dispreferred) tuples
            batch_size: Size of training batches
            
        Returns:
            batches: List of prepared batches
        """
        batches = []
        
        for i in range(0, len(preference_data), batch_size):
            batch_data = preference_data[i:i + batch_size]
            
            # Prepare inputs for preferred responses
            preferred_texts = []
            dispreferred_texts = []
            
            for prompt, preferred, dispreferred in batch_data:
                # Format as conversation for consistency with our SFT format
                preferred_text = f"User: {prompt}\n\nAssistant: {preferred}"
                dispreferred_text = f"User: {prompt}\n\nAssistant: {dispreferred}"
                
                preferred_texts.append(preferred_text)
                dispreferred_texts.append(dispreferred_text)
            
            # Tokenize both sets
            preferred_inputs = self.tokenizer(
                preferred_texts,
                padding=True,
                truncation=True,
                max_length=512,
                return_tensors="pt"
            ).to(self.device)
            
            dispreferred_inputs = self.tokenizer(
                dispreferred_texts,
                padding=True,
                truncation=True,
                max_length=512,
                return_tensors="pt"
            ).to(self.device)
            
            batch = {
                'preferred_ids': preferred_inputs['input_ids'],
                'preferred_mask': preferred_inputs['attention_mask'],
                'dispreferred_ids': dispreferred_inputs['input_ids'],
                'dispreferred_mask': dispreferred_inputs['attention_mask']
            }
            
            batches.append(batch)
        
        return batches
    
    def bradley_terry_loss(self, preferred_rewards, dispreferred_rewards):
        """
        Compute Bradley-Terry loss for preference learning
        This is the same loss used across TRPO, PPO, and GRPO
        
        Args:
            preferred_rewards: Rewards for preferred responses
            dispreferred_rewards: Rewards for dispreferred responses
            
        Returns:
            loss: Bradley-Terry loss encouraging preferred > dispreferred
        """
        # Compute log probability that preferred is better
        logits = preferred_rewards - dispreferred_rewards
        loss = -F.logsigmoid(logits).mean()
        
        return loss
    
    def train_step(self, batch):
        """
        Single training step on a batch of preference data
        
        Args:
            batch: Dictionary containing preference batch data
        """
        self.optimizer.zero_grad()
        
        # Forward pass for preferred responses
        preferred_rewards = self.model(
            batch['preferred_ids'], 
            batch['preferred_mask']
        )
        
        # Forward pass for dispreferred responses
        dispreferred_rewards = self.model(
            batch['dispreferred_ids'],
            batch['dispreferred_mask'] 
        )
        
        # Compute Bradley-Terry loss (same as used in TRPO/PPO/GRPO)
        loss = self.bradley_terry_loss(preferred_rewards, dispreferred_rewards)
        
        # Backward pass and optimization
        loss.backward()
        self.optimizer.step()
        
        # Calculate accuracy (how often preferred > dispreferred)
        accuracy = (preferred_rewards > dispreferred_rewards).float().mean()
        
        return {
            'loss': loss.item(),
            'accuracy': accuracy.item(),
            'preferred_reward_mean': preferred_rewards.mean().item(),
            'dispreferred_reward_mean': dispreferred_rewards.mean().item(),
            'reward_gap': (preferred_rewards - dispreferred_rewards).mean().item()
        }
    
    def train(self, preference_data, num_epochs=3, batch_size=8):
        """
        Train the reward model on preference data
        
        Args:
            preference_data: List of preference tuples
            num_epochs: Number of training epochs
            batch_size: Training batch size
        """
        print(f"Training reward model for {num_epochs} epochs...")
        
        # Prepare batches
        batches = self.prepare_preference_batch(preference_data, batch_size)
        
        for epoch in range(num_epochs):
            total_loss = 0
            total_accuracy = 0
            
            progress_bar = tqdm(batches, desc=f"Epoch {epoch+1}/{num_epochs}")
            
            for batch in progress_bar:
                metrics = self.train_step(batch)
                total_loss += metrics['loss']
                total_accuracy += metrics['accuracy']
                
                # Update progress bar
                progress_bar.set_postfix({
                    'loss': f"{metrics['loss']:.4f}",
                    'acc': f"{metrics['accuracy']:.3f}",
                    'gap': f"{metrics['reward_gap']:.3f}"
                })
            
            avg_loss = total_loss / len(batches)
            avg_accuracy = total_accuracy / len(batches)
            
            print(f"Epoch {epoch+1} - Loss: {avg_loss:.4f}, Accuracy: {avg_accuracy:.3f}")
    
    def evaluate_responses(self, prompts, responses):
        """
        Evaluate responses using the trained reward model
        This method will be used by TRPO, PPO, and GRPO optimizers
        
        Args:
            prompts: List of prompts
            responses: List of responses to evaluate
            
        Returns:
            rewards: Tensor of reward scores
        """
        self.model.eval()
        rewards = []
        
        with torch.no_grad():
            for prompt, response in zip(prompts, responses):
                # Format as conversation
                text = f"User: {prompt}\n\nAssistant: {response}"
                
                # Tokenize
                inputs = self.tokenizer(
                    text,
                    truncation=True,
                    max_length=512,
                    return_tensors="pt"
                ).to(self.device)
                
                # Get reward score
                reward = self.model(inputs['input_ids'], inputs['attention_mask'])
                rewards.append(reward.item())
        
        return torch.tensor(rewards)
    
    def save_model(self, path):
        """Save the trained reward model"""
        os.makedirs(path, exist_ok=True)
        torch.save(self.model.state_dict(), os.path.join(path, "reward_model.pt"))
        self.tokenizer.save_pretrained(path)
        print(f"Reward model saved to {path}")
    
    def load_model(self, path):
        """Load a trained reward model"""
        self.model.load_state_dict(torch.load(os.path.join(path, "reward_model.pt")))
        print(f"Reward model loaded from {path}")

# Example usage integrating with our existing classes
def create_reward_model_example():
    """
    Example of creating and training a reward model using our existing classes
    This reward model will be used consistently across TRPO, PPO, GRPO algorithms
    """
    
    # Use our existing PretrainedLLM class
    base_llm = PretrainedLLM(model_name="facebook/opt-350m")
    
    # Create and train SFT model (using our existing SupervisedFineTuner)
    sft = SupervisedFineTuner(base_llm, dataset_name="databricks/dolly-15k")
    processed_data = sft.prepare_data()
    peft_model = sft.setup_peft()
    adapter_path = sft.train(output_dir="sft_model", num_epochs=1)
    base_llm.load_adapter(adapter_path)
    
    # Create reward model using the same base architecture
    reward_model = RewardModel(base_llm)
    reward_trainer = RewardModelTrainer(reward_model)
    
    # Create preference dataset (simulating human preferences)
    test_prompts = [
        "Explain quantum computing to a 10-year-old",
        "How do I bake a chocolate cake?",
        "What is machine learning?",
        "Write a short story about friendship",
        "Explain gravity in simple terms"
    ]
    
    preference_data = reward_trainer.create_preference_dataset(sft, test_prompts)
    
    # Train the reward model (same training process for TRPO/PPO/GRPO)
    reward_trainer.train(preference_data, num_epochs=2)
    
    # Test the reward model
    print("\n=== Reward Model Evaluation ===")
    test_responses = [
        "Quantum computers are like magic calculators that can solve many problems at once!",
        "Quantum computing involves qubits and superposition states in quantum mechanical systems.",
        "Hey there! Imagine quantum computers as super-smart puzzle solvers that can try all solutions simultaneously!"
    ]
    
    prompt = "Explain quantum computing to a 10-year-old"
    rewards = reward_trainer.evaluate_responses([prompt] * 3, test_responses)
    
    for i, (response, reward) in enumerate(zip(test_responses, rewards)):
        print(f"\nResponse {i+1}: {response[:60]}...")
        print(f"Reward Score: {reward:.3f}")
    
    # Save the reward model for use in optimization algorithms
    reward_trainer.save_model("reward_model")
    
    return reward_model, reward_trainer


#### Understanding the RewardModel Class

The RewardModel class is fundamentally a neural network that takes a conversation (prompt + response) and outputs a single number representing how "good" or "helpful" that response is according to human preferences. Think of it as an automated judge that has learned to evaluate responses the way humans would. The architecture is brilliant in its simplicity yet powerful in its approach.

When we initialize the reward model, we start with an existing pretrained language model from our PretrainedLLM class. This is crucial because we want the reward model to understand language with the same sophistication as the models it will be evaluating. The initialization process does something very interesting - it takes the transformer backbone (all those attention layers that understand language) but removes the language modeling head that normally predicts the next token. Instead, it replaces this with a simple linear layer that outputs just one number - the reward score. This architectural change transforms a text generator into a text evaluator.

The reward head initialization is carefully done with small random weights and zero bias. This ensures that initially, the model doesn't have strong preferences and can learn from the human preference data without being biased toward any particular type of response. The total parameter count is printed to show how large the model is - in this case, it includes all the transformer parameters plus the small reward head.

The forward pass through the reward model is where the magic happens. When you give it a conversation, it processes the entire sequence through the transformer layers, just like a normal language model would. However, instead of using all the hidden states to predict next tokens, it focuses specifically on the final hidden state at the last meaningful position. This is important because the last token position contains the most comprehensive understanding of the entire conversation - it has "seen" and processed all the previous context.

The attention mask handling is crucial for practical implementation. In batched processing, sequences have different lengths, so we pad shorter sequences with special tokens. The attention mask tells us which tokens are real content versus padding. The code finds the last non-padding token for each sequence in the batch and extracts the hidden state at that position. This ensures we're always evaluating based on the actual end of the conversation, not padding tokens. Finally, this rich hidden representation is projected through the simple linear reward head to produce a single scalar score.

#### The RewardModelTrainer Class Deep Dive

The RewardModelTrainer class is where the reward model actually learns human preferences. The initialization is straightforward - it takes the reward model and sets up an optimizer to train it. The choice of AdamW optimizer with a small learning rate (1e-5) is deliberate because we're fine-tuning a large pretrained model and want stable, gradual learning.

The create_preference_dataset method is particularly interesting because it simulates the human preference collection process. In real RLHF implementations, humans would manually compare responses and indicate preferences. Here, we automate this by generating multiple responses to the same prompt using our SFT model with high temperature (0.8) to ensure diversity. The higher temperature makes the sampling more random, so we get varied responses that can be meaningfully compared.

For each prompt, the method generates several responses, then creates all possible pairwise comparisons between them. The _simple_quality_score function serves as our simulated human evaluator. It implements heuristics that reflect what humans typically prefer: interactive responses with questions, engaging language, appropriate length, and good analogies. While this is simplified compared to real human judgment, it captures important patterns that help train the reward model effectively.

The scoring system is quite thoughtful. Responses get points for containing questions (indicating interactivity), using engaging words like "imagine" or "amazing," having appropriate length (not too short or too long), and using analogy words that suggest good explanations. Responses that are too short are penalized because they're often unhelpful. This creates a ranking system where more engaging, helpful responses score higher than dry, technical ones.

Batch Processing and Training Mechanics
The prepare_preference_batch method handles the crucial task of converting preference data into trainable batches. This is where the Bradley-Terry model framework becomes operational. Each preference pair (preferred and dispreferred response) needs to be tokenized and formatted consistently. The method formats both responses as complete conversations, then tokenizes them with padding and truncation to ensure uniform batch sizes.

The batch structure is carefully designed with separate tokenized inputs for preferred and dispreferred responses, along with their attention masks. This allows the model to process both responses in the same forward pass, making training efficient. The maximum length of 512 tokens is a practical choice that balances computational efficiency with the ability to handle reasonably long conversations.

The bradley_terry_loss function implements the mathematical heart of preference learning. The Bradley-Terry model assumes that if humans prefer response A over response B, then the reward model should assign a higher score to A than to B. The loss function encourages this by computing the difference between preferred and dispreferred rewards, then using log-sigmoid to convert this into a probability. The negative log-sigmoid loss means we're maximizing the probability that preferred responses score higher than dispreferred ones.

#### Training Process and Optimization

The train_step method orchestrates a single training iteration. It performs forward passes through both preferred and dispreferred responses, computes the Bradley-Terry loss, and updates the model parameters. The accuracy calculation is particularly insightful - it measures how often the model correctly predicts that preferred responses should score higher than dispreferred ones. This gives us an immediate sense of how well the model is learning human preferences.

The metrics returned from each training step provide valuable insights into the learning process. The loss tells us how well the model is optimizing the preference ranking objective. The accuracy shows the percentage of correct preference predictions. The reward means for preferred and dispreferred responses help us understand if the model is learning to assign appropriate score ranges. The reward gap is especially important - it measures how well the model can distinguish between good and bad responses.

The main training loop in the train method coordinates everything together. It processes the preference data into batches, then iterates through multiple epochs of training. The progress bar provides real-time feedback on training metrics, helping us monitor whether the model is learning effectively. The epoch-level summaries give us a high-level view of training progress.

#### Evaluation and Practical Usage

The evaluate_responses method demonstrates how the trained reward model will be used in practice. During PPO or other RL optimization, we need to quickly evaluate many potential responses to guide the policy toward better outputs. This method takes prompts and responses, formats them consistently, and returns numerical scores that quantify quality according to learned human preferences.

The evaluation process puts the model in eval mode to disable dropout and other training-specific behaviors, ensuring consistent scoring. The no-grad context prevents gradient computation since we're only doing inference. Each prompt-response pair is formatted, tokenized, and passed through the reward model to get a score. These scores will later guide the RL optimization process.

The save and load methods handle model persistence, which is crucial for practical deployment. The reward model state dictionary contains all the learned parameters, while the tokenizer must be saved separately to ensure consistent text processing. This modular saving approach allows the reward model to be loaded and used independently of the original training setup.

The create_reward_model_example function demonstrates the complete end-to-end workflow. It shows how the reward model integrates with our existing classes - starting from a pretrained LLM, creating an SFT model, generating preference data, training the reward model, and finally evaluating its performance. This integration ensures that all components of our RLHF pipeline work together seamlessly.

The example evaluation at the end provides a concrete demonstration of how the reward model distinguishes between different response qualities. Technical responses should score lower than engaging, child-friendly explanations, and the trained model should reflect these learned preferences in its numerical scores. This completes the transformation from human preference data to an automated evaluation system that can guide large-scale optimization.



### Optimization Algorithms

#### Why these algorithms were needed?

Understanding why specialized optimization algorithms like TRPO, PPO, GRPO, and DPO became necessary requires diving deep into the fundamental challenges of policy optimization in reinforcement learning, especially when applied to the complex domain of language model alignment. The journey from basic policy gradient methods to sophisticated RLHF algorithms reveals a fascinating progression of mathematical insights and practical engineering solutions that address some of the most challenging problems in machine learning optimization.


At its core, policy optimization in reinforcement learning seeks to find the optimal policy $\pi^*$ that maximizes expected cumulative reward. The policy gradient theorem, first formalized by Sutton et al., provides the mathematical foundation for this optimization. For a parameterized policy $\pi_\theta$, the objective is to maximize:

$$J(\theta) = \mathbb{E}_{\tau \sim \pi_\theta}[R(\tau)]$$

where $\tau$ represents a trajectory (sequence of state-action pairs) and $R(\tau)$ is the cumulative reward. The policy gradient theorem tells us that:

$$\nabla_\theta J(\theta) = \mathbb{E}_{\tau \sim \pi_\theta}\left[\sum_{t=0}^T \nabla_\theta \log \pi_\theta(a_t|s_t) \hat{A}_t\right]$$

This equation is the policy gradient theorem formula, which is fundamental to policy optimization in reinforcement learning. It states that the gradient of the expected reward objective function equals the expected value of the sum of policy gradients at each time step, weighted by advantage estimates. In simpler terms, it tells us how to adjust the policy parameters to increase the probability of actions that lead to better-than-expected outcomes (positive advantage) and decrease the probability of actions that lead to worse-than-expected outcomes (negative advantage). This provides the mathematical foundation for how reinforcement learning algorithms can systematically improve a policy through gradient-based optimization.

This elegant formulation suggests we can optimize policies by following the gradient of expected rewards. However, this seemingly straightforward approach conceals profound computational and statistical challenges that become especially acute when applied to language models.

The basic REINFORCE algorithm implements this gradient estimate directly, but it suffers from catastrophically high variance. In language models, where a single trajectory might involve generating hundreds of tokens, each with tens of thousands of possible choices, the variance of gradient estimates becomes so large that learning becomes practically impossible. The fundamental issue lies in the credit assignment problem: when a complete response receives a reward (positive or negative), which of the hundreds of preceding token choices actually contributed to that outcome?

Consider our quantum computing explanation scenario. If our model generates: "Hey there! Imagine quantum computers as magical calculators that can solve many problems simultaneously by existing in multiple states at once, kind of like a spinning coin that's both heads and tails until it stops!" and receives a high reward score, the basic policy gradient has no principled way to determine whether the reward should be attributed to the engaging opening ("Hey there!"), the analogy choice ("magical calculators"), the technical accuracy ("multiple states at once"), or the relatable comparison ("spinning coin"). This attribution ambiguity leads to gradient estimates with enormous variance, making stable learning nearly impossible.

##### The Challenge of Sample Efficiency

Language model training presents unique sample efficiency challenges that traditional RL algorithms weren't designed to handle. Unlike robotics or game-playing scenarios where millions of interactions might be feasible, generating text samples from large language models is computationally expensive. Each forward pass through a model with billions of parameters requires significant computation, and generating diverse, high-quality responses for preference learning demands even more resources.

Furthermore, the evaluation bottleneck compounds this efficiency problem. While traditional RL environments provide immediate reward signals (hitting a wall gives instant negative feedback), language model evaluation requires either expensive human annotation or carefully trained reward models. This sparse reward structure means we must extract maximum learning signal from every expensive sample we generate.

The mathematical challenge becomes clear when we examine the variance of policy gradient estimators. For a policy gradient estimate $\hat{g}$, the variance scales approximately as:

$$\text{Var}[\hat{g}] \propto \frac{1}{N} \sum_{t=0}^T \text{Var}[R_t]$$

where $N$ is the sample size and $T$ is the trajectory length. In language models, $T$ can be hundreds of tokens, and $\text{Var}[R_t]$ can be enormous due to the discrete, high-dimensional action space. Traditional variance reduction techniques like baselines help but aren't sufficient for the scale of the problem.

##### On-Policy vs. Off-Policy Learning: A Critical Distinction

The distinction between on-policy and off-policy learning becomes crucial when we consider the practical constraints of language model training. On-policy methods require that training data comes from the current policy being optimized. Every time we update our model parameters, all previously collected data becomes "stale" because it was generated from a different policy. This means we must constantly generate fresh samples, making on-policy methods sample-inefficient but ensuring that our gradient estimates remain unbiased.

Mathematically, on-policy methods maintain the condition:

$$\mathbb{E}_{\tau \sim \pi_{\theta_{\text{current}}}}[\nabla_\theta \log \pi_\theta(a_t|s_t) \hat{A}_t]$$

This expectation is taken with respect to the current policy, ensuring unbiased gradient estimates but requiring constant sample generation.

Off-policy methods, conversely, can reuse data collected from previous policy versions through importance sampling corrections:

$$\mathbb{E}_{\tau \sim \pi_{\theta_{\text{old}}}}\left[\frac{\pi_{\theta_{\text{current}}}(a_t|s_t)}{\pi_{\theta_{\text{old}}}(a_t|s_t)} \nabla_\theta \log \pi_\theta(a_t|s_t) \hat{A}_t\right]$$



The importance sampling ratio $\frac{\pi_{\theta_{\text{current}}}(a_t|s_t)}{\pi_{\theta_{\text{old}}}(a_t|s_t)}$ theoretically corrects for the distribution mismatch, but in practice, this ratio can become extremely large or small, leading to high variance that negates the sample efficiency benefits.

In language models, this trade-off becomes particularly stark. The computational cost of generating samples is so high that off-policy methods seem attractive, but the high-dimensional discrete action space makes importance sampling corrections notoriously unstable. A single token change can dramatically shift the probability of entire sequences, leading to importance weights that vary by orders of magnitude.

##### Why Naive Optimization Fails

Perhaps the most fundamental challenge that necessitated specialized algorithms is the stability problem inherent in direct policy optimization. Language models exist in extremely high-dimensional parameter spaces—modern models have billions of parameters—and the loss landscape is notoriously non-convex with many local minima and saddle points.

When we apply basic policy gradients to language models, small parameter updates can lead to catastrophic changes in policy behavior. This happens because the probability of generating any specific sequence depends on the product of probabilities across all tokens in that sequence:

$$P_\theta(y|x) = \prod_{t=1}^T \pi_\theta(y_t|x, y_{<t})$$

This equation represents the probability of generating a complete response sequence (y) given a prompt (x), which equals the product of probabilities for each individual token choice. It shows that even small changes in individual token probabilities can compound exponentially across the sequence length, potentially causing dramatic shifts in the model's overall behavior. This mathematical reality explains why naive optimization methods can lead to unstable training when applied to language models.

Even small changes to individual token probabilities can compound exponentially across the sequence, leading to dramatic shifts in the overall policy. In our quantum computing example, a small parameter change that slightly increases the probability of technical terms might completely alter the model's tendency to use child-friendly language, undermining months of training.

This sensitivity is mathematically captured by the condition number of the optimization problem. For policy optimization, we're essentially trying to solve:

$$\theta^* = \arg\max_\theta \mathbb{E}_{\tau \sim \pi_\theta}[R(\tau)]$$

The curvature of this objective function—measured by its Hessian—can vary dramatically across the parameter space. Regions where the policy changes rapidly with parameter adjustments correspond to high curvature, making gradient-based optimization unstable without careful step size control.


#####  Why This Matters for AI Safety

The development of these optimization algorithms isn't just a technical curiosity—it represents crucial progress toward building AI systems that can be reliably aligned with human values and preferences. The mathematical rigor required to make these algorithms work reveals deep insights about the nature of learning from human feedback and the challenges of encoding complex human preferences into computational systems.

The stability guarantees provided by trust region methods ensure that AI systems won't suddenly exhibit catastrophic behavioral changes during training. The sample efficiency improvements enable learning from limited human feedback, making it feasible to train large-scale systems without requiring prohibitive amounts of human annotation. The theoretical foundations provide confidence that these systems will behave predictably and safely as they scale to larger models and more complex tasks.

As we dive into the specific algorithms in the following sections, we'll see how each method addresses these fundamental challenges through different mathematical approaches and engineering trade-offs. The progression from basic policy gradients to sophisticated RLHF algorithms represents one of the most important developments in modern machine learning, with implications that extend far beyond language model training to the broader challenge of building AI systems that are helpful, harmless, and honest.

#### Trust Region Policy Optimization

<img src="assets/trpo.png" width=900>


Trust Region Policy Optimization (TRPO), introduced by Schulman et al. in 2015, represents a pivotal breakthrough in reinforcement learning that directly addresses the fundamental instability problems plaguing vanilla policy gradient methods. To understand TRPO's revolutionary approach, let's revisit our quantum computing explanation scenario and see how it transforms the chaotic landscape of policy optimization into a controlled, mathematically principled learning process.

**The Core Problem TRPO Solves**

Imagine our language model is like a student learning to explain quantum computing to children. In vanilla policy gradient methods, each update is like giving the student feedback and saying "adjust your teaching style," but without any constraints on how drastically they should change. The student might completely abandon their previous approach overnight—going from using simple analogies to suddenly employing advanced mathematical notation—creating catastrophic instability in learning.

The fundamental issue lies in the relationship between policy updates and performance guarantees. When we update our policy from $\pi_{\theta_{\text{old}}}$ to $\pi_{\theta}$, vanilla policy gradient methods provide no guarantees about how the new policy will perform. A small change in parameters might lead to dramatically different token probabilities, and since language generation depends on the product of probabilities across all tokens:

$$P_\theta(y|x) = \prod_{t=1}^T \pi_\theta(y_t|x, y_{<t})$$

Even tiny parameter changes can compound exponentially, leading to completely different responses. In our quantum explanation example, a small increase in the probability of technical terms early in the sequence might cascade through the entire response, transforming a child-friendly explanation into an incomprehensible academic discourse.

**The Trust Region Insight**

TRPO's breakthrough insight is remarkably intuitive: policy gradient estimates are only locally valid. Just as a map of your neighborhood won't help you navigate across the country, gradient estimates computed at policy $\pi_{\theta_{\text{old}}}$ are only reliable in a small "neighborhood" around that policy. Step too far away, and the gradient information becomes meaningless or even misleading.

This leads to TRPO's central principle: constrain policy updates to remain within a "trust region" where our gradient estimates are reliable. In our teaching analogy, this means allowing our student to refine their explanation techniques gradually, ensuring they don't abandon effective strategies too quickly while still making steady progress toward better teaching.

**Mathematical Foundation: The Surrogate Objective**

TRPO begins with the policy gradient theorem but reformulates it using importance sampling to create a surrogate objective that can be optimized using data from the old policy. The standard policy gradient objective:

$$\eta(\pi) = \mathbb{E}_{\tau \sim \pi}\left[\sum_{t=0}^T R_t\right]$$

is transformed into the surrogate objective:

$$L^{\pi_{\theta_{\text{old}}}}(\pi_\theta) = \mathbb{E}_{\tau \sim \pi_{\theta_{\text{old}}}}\left[\sum_{t=0}^T \frac{\pi_\theta(a_t|s_t)}{\pi_{\theta_{\text{old}}}(a_t|s_t)} A^{\pi_{\theta_{\text{old}}}}(s_t, a_t)\right]$$

where:
- $\frac{\pi_\theta(a_t|s_t)}{\pi_{\theta_{\text{old}}}(a_t|s_t)}$ is the importance sampling ratio
- $A^{\pi_{\theta_{\text{old}}}}(s_t, a_t)$ is the advantage function under the old policy
- The expectation is taken over trajectories from the old policy

This formulation is brilliant because it allows us to estimate policy improvements using data we've already collected, rather than requiring new samples after each parameter update.

**The Advantage Function in Language Models**

The advantage function $A^\pi(s,a)$ measures how much better taking action $a$ in state $s$ is compared to the average action the policy would take. In our quantum explanation context:

$$A^\pi(\text{state}, \text{action}) = Q^\pi(\text{state}, \text{action}) - V^\pi(\text{state})$$

For example:
- **State**: "Explain quantum computing to a 10-year-old: Think of"
- **Action A**: "superposition" (technical term)
- **Action B**: "magic" (child-friendly term)

If $A^\pi(\text{state}, \text{"superposition"}) = -2.3$ and $A^\pi(\text{state}, \text{"magic"}) = +1.8$, this tells us that choosing "magic" leads to significantly better outcomes than the average action, while "superposition" performs worse than average.

**The Trust Region Constraint**

The key innovation of TRPO is constraining the KL divergence between consecutive policies:

$$\mathbb{E}_{s \sim \rho_{\pi_{\theta_{\text{old}}}}}[D_{KL}(\pi_{\theta_{\text{old}}}(\cdot|s) \| \pi_\theta(\cdot|s))] \leq \delta$$

where:
- $\rho_{\pi_{\theta_{\text{old}}}}$ is the state visitation distribution under the old policy
- $\delta$ is the trust region size (typically 0.01 or 0.02)
- $D_{KL}$ is the Kullback-Leibler divergence

This constraint ensures that the new policy doesn't deviate too dramatically from the old one. In KL divergence terms, if two policies are identical, $D_{KL} = 0$. As policies become more different, KL divergence increases, providing a natural measure of policy similarity.

For our quantum explanation model, this means we can't suddenly switch from a child-friendly communication style to academic jargon. The constraint forces gradual refinement: improving word choices, adjusting analogies, and enhancing engagement while maintaining the overall communication approach.

**The Constrained Optimization Problem**

TRPO formulates policy optimization as a constrained optimization problem:

$$\underset{\theta}{\text{maximize}} \quad \mathbb{E}_{\tau \sim \pi_{\theta_{\text{old}}}}\left[\sum_{t=0}^T \frac{\pi_\theta(a_t|s_t)}{\pi_{\theta_{\text{old}}}(a_t|s_t)} A^{\pi_{\theta_{\text{old}}}}(s_t, a_t)\right]$$

$$\text{subject to} \quad \mathbb{E}_{s \sim \rho_{\pi_{\theta_{\text{old}}}}}[D_{KL}(\pi_{\theta_{\text{old}}}(\cdot|s) \| \pi_\theta(\cdot|s))] \leq \delta$$

This is a constrained optimization problem where we want to maximize expected improvement while staying within the trust region. The mathematical beauty lies in how this formulation provides theoretical guarantees about monotonic improvement.

**Theoretical Guarantees: The Policy Performance Bound**

TRPO's theoretical foundation rests on a crucial theorem that provides a lower bound on policy performance. The theorem states that:

$$\eta(\pi) \geq L^{\pi_{\text{old}}}(\pi) - \frac{4\epsilon\gamma}{(1-\gamma)^2}\alpha^2$$

where:
- $\eta(\pi)$ is the true performance of policy $\pi$
- $L^{\pi_{\text{old}}}(\pi)$ is the surrogate objective
- $\epsilon$ is the maximum advantage magnitude
- $\alpha = \max_s D_{TV}(\pi_{\text{old}}(\cdot|s) \| \pi(\cdot|s))$ measures policy difference
- $D_{TV}$ is the total variation distance

This bound guarantees that if we improve the surrogate objective while keeping the policy change small, we're guaranteed to improve the true policy performance. This is the mathematical foundation that makes TRPO's trust region approach theoretically sound.

**From Theory to Practice: The Conjugate Gradient Solution**

The constrained optimization problem is theoretically elegant but computationally challenging. Solving it exactly would require computing second-order derivatives (the Hessian) of the KL constraint, which is prohibitively expensive for large language models with billions of parameters.

TRPO's practical brilliance lies in its approximation method. It linearizes the objective and uses a quadratic approximation of the constraint:

1. **Linear approximation of objective**: $L^{\pi_{\theta_{\text{old}}}}(\theta) \approx L^{\pi_{\theta_{\text{old}}}}(\theta_{\text{old}}) + g^T(\theta - \theta_{\text{old}})$

2. **Quadratic approximation of constraint**: $\mathbb{E}[D_{KL}] \approx \frac{1}{2}(\theta - \theta_{\text{old}})^T H (\theta - \theta_{\text{old}})$

where $g$ is the policy gradient and $H$ is the Hessian of the KL divergence.

This leads to the closed-form solution:

$$\theta_{\text{new}} = \theta_{\text{old}} + \sqrt{\frac{2\delta}{g^T H^{-1} g}} H^{-1} g$$

The term $H^{-1} g$ is the "natural gradient" direction, and the scaling factor ensures we stay within the trust region.

**Conjugate Gradient Algorithm**

Computing $H^{-1}$ directly is still too expensive, so TRPO uses the conjugate gradient algorithm to solve $Hx = g$ iteratively. This requires only matrix-vector products $Hv$, which can be computed efficiently using automatic differentiation:

$$Hv = \nabla_\theta \left(\left(\nabla_\theta \mathbb{E}[D_{KL}]\right)^T v\right)$$

The conjugate gradient algorithm finds the solution $x = H^{-1}g$ without explicitly computing the inverse matrix, making the computation feasible for large neural networks.

**Line Search and Backtracking**

Even with the trust region constraint, TRPO includes an additional safety mechanism: backtracking line search. After computing the proposed update direction, TRPO tests the actual policy improvement and backs off if necessary:

1. Compute proposed update: $\theta_{\text{proposed}} = \theta_{\text{old}} + \alpha \sqrt{\frac{2\delta}{g^T H^{-1} g}} H^{-1} g$
2. Check KL constraint: $D_{KL}(\pi_{\theta_{\text{old}}}, \pi_{\theta_{\text{proposed}}}) \leq \delta$
3. Check improvement: $\eta(\pi_{\theta_{\text{proposed}}}) > \eta(\pi_{\theta_{\text{old}}})$
4. If either fails, reduce $\alpha$ and try again

**TRPO in Language Model Context**

When applied to our quantum explanation model, TRPO's process looks like this:

1. **Current State**: Model generates technical explanations
2. **Collect Trajectories**: Generate multiple quantum explanations using current policy
3. **Compute Advantages**: Reward model scores indicate which responses are better
4. **Trust Region Update**: Adjust parameters to increase probability of good responses while staying close to current policy
5. **Verification**: Ensure new policy hasn't changed too dramatically

For example, if the current policy generates: "Quantum computers utilize quantum superposition..." and this receives a low reward, TRPO would gradually increase the probability of more engaging openings like "Hey there! Imagine..." without completely abandoning the model's existing knowledge.

**Why TRPO Was Groundbreaking**

TRPO represented a paradigm shift in several key ways:

1. **Theoretical Guarantees**: Unlike previous methods, TRPO provides mathematical guarantees about monotonic improvement, ensuring that each update makes the policy better (or at least no worse).

2. **Sample Efficiency**: By reusing data from the old policy through importance sampling, TRPO achieved better sample efficiency than on-policy methods that discard old data.

3. **Stability**: The trust region constraint prevented the catastrophic policy collapses that plagued vanilla policy gradient methods.

4. **Generality**: TRPO worked across diverse domains, from robotics to game playing to (eventually) language modeling.

**Limitations and Computational Challenges**

Despite its theoretical elegance, TRPO has significant practical limitations:

1. **Computational Complexity**: The conjugate gradient algorithm and line search make each update computationally expensive, requiring multiple forward and backward passes.

2. **Hyperparameter Sensitivity**: The trust region size $\delta$ requires careful tuning. Too small, and learning is slow; too large, and the approximations break down.

3. **Implementation Complexity**: TRPO requires implementing conjugate gradient, Hessian-vector products, and line search, making it significantly more complex than simpler alternatives.

4. **Scalability Issues**: For very large language models, even the conjugate gradient approach can be prohibitively expensive.

**The Path to PPO**

TRPO's limitations directly motivated the development of Proximal Policy Optimization (PPO), which we'll explore next. PPO achieves similar stability guarantees with a much simpler implementation, making it the preferred choice for most practical applications, including large-scale language model training.

However, understanding TRPO is crucial because:
- It established the theoretical foundations for trust region methods
- It demonstrated the importance of constraining policy updates
- It introduced key concepts like surrogate objectives and natural gradients
- It provided the mathematical intuition that guides more practical algorithms

In our quantum computing education example, TRPO would gradually transform our model from generating technical academic responses to producing engaging, child-friendly explanations through a series of constrained, theoretically guaranteed improvements. Each step would maintain the model's core knowledge while progressively aligning its communication style with human preferences for age-appropriate education.

The mathematical rigor of TRPO established trust region methods as a cornerstone of modern reinforcement learning, providing the theoretical foundation upon which practical algorithms like PPO build their simplified yet effective approaches to policy optimization.

In policy gradient algorithms, update steps computed at any specific policy $\pi_{\theta}$ are only really "predictive" in the neighborhood of $\theta_t$. That is, it is probable that updates outside of this neighborhood may not contain any predictive value at all. Intuitively, you may then think of constraining updates so that they do stay in the vicinity of our current policy.

Since the policy is a probability distribution over (state, action) pairs, we refer to it as the "policy space". Trust region methods (Kakade, 2001; Schulman et al., 2015a; 2017) aim to restrict how far successive policies are allowed to deviate. 

In Trust Region Policy Optimization(Schulman et al., 2015a), this is achieved by minimizing the KL divergence between successive policies on the optimization trajectory. Let $\hat{A_{\pi}}(s_t, a_t)$ be the estimated advantage function where the agent picks action $a$ at time $t$ given he is at state $s$ while following a policy $\pi_{\theta}$. Let $\pi(a_t|s_t)$ be the old policy where we take action $a$ given we are in state $s$ at time $t$. $\pi_{\theta}(a_t|s_t)$ is our current (parameterized) policy which we seek to update.

Define the optimization problem as:
$$\underset{\theta}{\max}\ \mathbb{E}_{(s_t, a_t)\sim \pi} \left[\frac{\pi_{\theta}(a_t|s_t)}{\pi(a_t|s_t)}\hat{A_{\pi}}(s_t, a_t)\right]\\ \\
\text{subject to } D_{KL}(\pi_{\theta}(\cdot|s)||\pi_{\theta}(\cdot|s))\leq\delta, \ \forall s
$$
The "subject to" line is essentially an assumption that we will have to improve. It is saying that our optimization problem will abide by the goal of having the KL divergence between the two policies becoming less than some small $\delta$.

This theoretical update is not easy to compute. Thus, TRPO makes approximations by reformulating both the loss function $\mathcal{L}(\theta_k, \theta)$ and the KL divergence $D_KL(\theta||\theta_k)$ to give an easier-to-compute approximation of the objective. Furthermore, this approximate objective is then able to be solved using Lagrangian duality to yield the update:
$$\theta_{k+1}=\theta_k+\alpha^j\sqrt{\frac{2\delta}{g^TH^{-1}g}}H^{-1}g$$
Since the Hessian inverse $H^{-1}$ is expensive to compute, TRPO utilizes the conjugate gradient algorithm to solve $Hx=g$ (or $x=H^{-1}g$) which requires a function for computing $Hx$ instead of computing and storing the entire matrix $H$.

In [ ]:
# CODE IMPLEMENTATION TODO

CODE EXPLANATION TODO

#### Proximal Policy Optimization

Proximal Policy Optimization (PPO), introduced by Schulman et al. in 2017, represents one of the most significant practical breakthroughs in reinforcement learning. While TRPO established the theoretical foundations for trust region methods, PPO achieved something equally important: making these powerful ideas simple, efficient, and implementable at scale. To understand PPO's revolutionary impact, let's continue with our quantum computing explanation scenario and see how it transforms TRPO's complex mathematics into an elegant, practical solution.



**PPO's Brilliant Insight: First-Order Trust Regions**

PPO's breakthrough came from a deceptively simple question: "What if we could achieve TRPO's stability guarantees using only first-order gradient information?" Instead of computing complex trust region constraints exactly, PPO approximates them using a much simpler clipping mechanism that achieves similar goals with dramatically reduced computational cost.

The key insight is that we don't need to solve the constrained optimization problem exactly—we just need to prevent policy updates from being too large. PPO achieves this through a clever surrogate objective that automatically limits the magnitude of policy changes.

**The PPO-Clip Objective**

PPO replaces TRPO's complex constrained optimization with a simple clipped surrogate objective. For each state-action pair, PPO computes the probability ratio:

$$r_t(\theta) = \frac{\pi_\theta(a_t|s_t)}{\pi_{\theta_{\text{old}}}(a_t|s_t)}$$

This ratio measures how much the new policy $\pi_\theta$ differs from the old policy $\pi_{\theta_{\text{old}}}$ for a specific action. When $r_t(\theta) = 1$, the policies are identical for that action. When $r_t(\theta) > 1$, the new policy assigns higher probability to the action, and when $r_t(\theta) < 1$, it assigns lower probability.

The clipped objective is then defined as:

$$L^{\text{CLIP}}(\theta) = \mathbb{E}_t\left[\min\left(r_t(\theta) A_t, \text{clip}(r_t(\theta), 1-\epsilon, 1+\epsilon) A_t\right)\right]$$

where:
- $A_t$ is the advantage function at time $t$
- $\epsilon$ is the clipping parameter (typically 0.1 or 0.2)
- $\text{clip}(x, a, b)$ constrains $x$ to the interval $[a, b]$

**Case 1: Positive Advantage ($A_t > 0$)**
When an action leads to better-than-expected outcomes, we want to increase its probability. However, PPO prevents us from increasing it too much:

- If $r_t(\theta) < 1 + \epsilon$: The objective becomes $r_t(\theta) A_t$, encouraging normal updates
- If $r_t(\theta) > 1 + \epsilon$: The objective becomes $(1 + \epsilon) A_t$, preventing excessive increases

For example, if our model discovers that starting with "Hey there!" leads to much better engagement scores, PPO will increase the probability of this opening, but not so much that it dominates all other possibilities.

**Case 2: Negative Advantage ($A_t < 0$)**
When an action leads to worse-than-expected outcomes, we want to decrease its probability, but again with limits:

- If $r_t(\theta) > 1 - \epsilon$: The objective becomes $r_t(\theta) A_t$, encouraging normal updates
- If $r_t(\theta) < 1 - \epsilon$: The objective becomes $(1 - \epsilon) A_t$, preventing excessive decreases

If our model finds that using technical jargon like "superposition" in child explanations gets poor scores, PPO will decrease its probability, but won't eliminate it entirely in a single update.

The clipping mechanism is mathematically elegant because it creates a pessimistic (conservative) objective. The $\min$ operation always chooses the smaller of the two terms, preventing overoptimistic updates. This conservatism provides implicit trust region enforcement without requiring complex second-order computations.

To understand why this works, consider the relationship between the clipped objective and policy performance. When we clip the importance sampling ratios, we're effectively limiting how much the policy can change in a single update. This prevents the policy from making large, potentially destabilizing jumps while still allowing meaningful progress toward better performance.

Now let's walk through the complete PPO workflow step by step, following the training pipeline that makes this objective functional:

<img src="assets/ppo2.png">

<img src="assets/ppo.png">

The PPO training process follows a systematic workflow that transforms our quantum explanation model through iterative improvements. Let's trace through each component of this pipeline:

##### Step 1: Trajectory Collection with Current Policy (or Current Actor)

The first step in each PPO iteration involves generating multiple quantum explanations using our current policy $\pi_{\theta_{\text{old}}}$. This is where our model produces diverse responses that will serve as training data for the next update.

**Process:**
For our quantum computing explanation task, we present the model with prompts like "Explain quantum computing to a 10-year-old" and generate multiple responses:

- **Response 1**: "Quantum computers are special machines that use quantum bits..."
- **Response 2**: "Hey there! Imagine quantum computers as magical calculators..."
- **Response 3**: "Think of quantum computing like having a coin that spins..."

**Mathematical Framework:**
Each trajectory $\tau = (s_0, a_0, s_1, a_1, \ldots, s_T, a_T)$ represents a complete conversation where:
- $s_t$ is the context (prompt + tokens generated so far)
- $a_t$ is the next token chosen
- The policy $\pi_{\theta_{\text{old}}}(a_t|s_t)$ determines token selection probabilities

**Key Implementation Details:**
- We use sampling (not greedy decoding) to ensure diversity
- Temperature controls exploration vs exploitation
- We collect multiple trajectories per prompt to capture response variety
- Each trajectory includes the complete conversation history

##### Step 2: Reward Model Evaluation (or Current Actor)

Once we have collected trajectories, each complete response is evaluated using our trained reward model to get scalar reward scores.

**Reward Assignment Process:**
Our reward model $r_\theta(x, y)$ evaluates each prompt-response pair:

For our quantum explanation examples:
- **Response 1**: Technical explanation → Reward: 3.2
- **Response 2**: Engaging with analogies → Reward: 7.8  
- **Response 3**: Simple analogy → Reward: 6.5

**Mathematical Foundation:**
The reward model outputs scalar scores based on human preferences learned during reward model training:
$$r_\theta(x, y) = \text{RewardModel}(x \oplus y)$$

where $x \oplus y$ represents the concatenated prompt and response.

**Sparse Reward Challenge:**
Unlike traditional RL where rewards occur throughout the trajectory, language models receive rewards only at sequence completion. This creates the credit assignment problem that PPO's advantage estimation must solve.

##### Step 3: Value Function (or Critic Function) Estimation and Advantage Computation

The value function $V^\pi(s)$ estimates the expected future reward from any given state, while advantages $A^\pi(s,a)$ measure how much better a specific action is compared to the average.

**Value Function Role:**
$$V^\pi(s_t) = \mathbb{E}_\pi\left[\sum_{k=t}^T \gamma^{k-t} r_k \mid s_t\right]$$

For our quantum explanation:
- $V("Explain quantum computing: ")$ = 5.2 (expected reward for this starting state)
- $V("Explain quantum computing: Hey there!")$ = 7.1 (higher expectation after engaging start)

**Generalized Advantage Estimation (GAE)**

PPO uses GAE to compute advantages with an optimal bias-variance trade-off:

$$\hat{A}_t^{\text{GAE}(\gamma,\lambda)} = \sum_{l=0}^{\infty} (\gamma\lambda)^l \delta_{t+l}$$

where $\delta_t = r_t + \gamma V(s_{t+1}) - V(s_t)$ is the temporal difference error.

**GAE Parameter Impact:**
- $\lambda = 0$: Uses only 1-step TD estimates (low variance, higher bias)
- $\lambda = 1$: Uses full Monte Carlo returns (high variance, no bias)
- $\lambda = 0.95$ (typical): Balances bias and variance effectively

**Advantage Interpretation in Language Models:**
- Positive advantage: This token choice leads to better outcomes than average
- Negative advantage: This token choice leads to worse outcomes than average
- Zero advantage: This token choice is about average

Example for our quantum explanation:
- $A("Explain quantum: ", \text{"Hey"}) = +1.8$ (engaging start is better than average)
- $A("Explain quantum: ", \text{"Quantum"}) = -0.9$ (technical start is worse than average)

##### Step 4: Policy Optimization with Clipped Objective

Now we reach the heart of PPO: the clipped surrogate objective that enables stable policy improvements.

**The Clipped Surrogate Loss:**
$$L^{\text{CLIP}}(\theta) = \mathbb{E}_t\left[\min\left(r_t(\theta) A_t, \text{clip}(r_t(\theta), 1-\epsilon, 1+\epsilon) A_t\right)\right]$$

where $r_t(\theta) = \frac{\pi_\theta(a_t|s_t)}{\pi_{\theta_{\text{old}}}(a_t|s_t)}$ is the probability ratio.

**Clipping Mechanism Deep Dive:**

**Scenario 1: Positive Advantage (Good Actions)**
When $A_t > 0$, we want to increase the probability of this action:

```
If r_t(θ) ≤ 1 + ε: Use r_t(θ)A_t (normal update)
If r_t(θ) > 1 + ε: Use (1 + ε)A_t (clip to prevent over-optimization)
```

In our quantum example, if "Hey there!" receives positive advantage, PPO increases its probability but prevents it from dominating completely.

**Scenario 2: Negative Advantage (Bad Actions)**  
When $A_t < 0$, we want to decrease the probability of this action:

```
If r_t(θ) ≥ 1 - ε: Use r_t(θ)A_t (normal update)
If r_t(θ) < 1 - ε: Use (1 - ε)A_t (clip to prevent over-penalization)
```

If "superposition" receives negative advantage in child explanations, PPO decreases its probability gradually rather than eliminating it entirely.

**Why Clipping Works:**
The clipping creates a pessimistic (conservative) objective that prevents policy collapse. The $\min$ operation always chooses the more conservative update, ensuring stability.

##### Step 5: Value Function Training

Parallel to policy optimization, PPO trains the value function to better estimate state values for future advantage computations.

**Value Function Loss with Clipping:**
$$L^{VF}(\theta) = \mathbb{E}_t\left[\max\left((V_\theta(s_t) - V_t^{\text{targ}})^2, (\text{clip}(V_\theta(s_t), V_{\text{old}} - \epsilon, V_{\text{old}} + \epsilon) - V_t^{\text{targ}})^2\right)\right]$$

**Why Clip the Value Function?**
Just as we clip policy updates, we clip value function updates to prevent large jumps that could destabilize advantage estimation. This ensures that our critic (value function) learns smoothly alongside our actor (policy).

**Target Computation:**
The target values $V_t^{\text{targ}}$ are computed using the GAE returns:
$$V_t^{\text{targ}} = \hat{A}_t + V(s_t)$$

This provides supervised targets for value function regression.

##### Step 6: Entropy Regularization

The entropy bonus encourages exploration by preventing the policy from becoming too deterministic:

$$S[\pi_\theta](s) = \mathbb{E}_{a \sim \pi_\theta(\cdot|s)}\left[-\log \pi_\theta(a|s)\right]$$

**Entropy in Language Models:**
For our quantum explanation model, entropy ensures response diversity:
- High entropy: Model considers many possible continuations
- Low entropy: Model strongly prefers specific continuations  

**Practical Impact:**
Without entropy regularization, our model might converge to generating identical "optimal" responses, losing the ability to adapt to different contexts or user preferences.

##### Step 7: Combined Loss Optimization

All three components combine in the final PPO objective:
$$L(\theta) = L^{\text{CLIP}}(\theta) - c_1 L^{VF}(\theta) + c_2 S[\pi_\theta](s)$$

**Coefficient Balancing:**
- $c_1 = 0.5$: Balances policy and value learning
- $c_2 = 0.01$: Provides gentle exploration pressure
- These ratios ensure policy optimization dominates while maintaining value function accuracy and exploration

**Multiple Epochs per Batch:**
PPO performs 3-10 gradient updates on the same batch of data, dramatically improving sample efficiency. The clipping mechanism prevents over-optimization during these multiple passes.

##### Step 8: Iterative Improvement

The entire process repeats iteratively:

**Iteration 1:**
- Collect trajectories from current policy
- Compute rewards and advantages  
- Update policy with clipped objective
- Result: Slightly better quantum explanations

**Iteration 2:**
- Use improved policy to collect new trajectories
- Rewards should generally increase due to better responses
- Continue optimization
- Result: More engaging, child-friendly explanations

**Iteration N:**
- Policy has learned to generate consistently high-quality responses
- Balances scientific accuracy with age-appropriate communication
- Maintains diversity while optimizing for human preferences

**Convergence Pattern:**
Our quantum explanation model progressively evolves:

1. **Initial**: Technical, dry explanations
2. **Early training**: Begins using simpler language
3. **Mid training**: Incorporates analogies and examples
4. **Late training**: Masters engaging, interactive communication
5. **Converged**: Consistently produces helpful, harmless, honest explanations

**Example Evolution Sequence:**

**Iteration 1**: "Quantum computers use qubits in superposition states..."  
**Reward**: 3.1 → **Update**: Increase probability of simpler terms

**Iteration 50**: "Quantum computers are like special calculators that can try multiple solutions..."  
**Reward**: 6.4 → **Update**: Increase probability of interactive elements  

**Iteration 200**: "Hey there! Imagine quantum computers as magical problem-solvers. What's your favorite type of puzzle?"  
**Reward**: 8.8 → **Update**: Fine-tune for even better engagement

**In essence:**

When applied to our quantum explanation model, PPO's training process involves:

1. **Trajectory Collection**: Generate multiple quantum explanations using the current policy
2. **Advantage Computation**: Use the reward model and value function to compute advantages for each token choice
3. **Policy Updates**: Apply the clipped objective to improve token selection probabilities
4. **Value Updates**: Train the critic to better estimate state values
5. **Multiple Epochs**: Repeat updates multiple times on the same batch (typically 3-10 epochs)


**Why PPO Works So Well**

PPO's success stems from several key advantages:

**1. Computational Efficiency**
PPO requires only first-order gradient information, making it 10-100 times faster than TRPO while achieving comparable performance. This efficiency enables training on large-scale language models.

**2. Implementation Simplicity**
The clipping mechanism is straightforward to implement and debug, unlike TRPO's complex conjugate gradient procedures. This simplicity has made PPO the de facto standard for many RL applications.

**3. Hyperparameter Robustness**
PPO works well with a standard set of hyperparameters across diverse tasks:
- Clipping parameter $\epsilon = 0.2$
- Value function coefficient $c_1 = 0.5$
- Entropy coefficient $c_2 = 0.01$
- GAE parameter $\lambda = 0.95$

**4. Sample Efficiency**
By performing multiple gradient updates on the same batch of data (typically 3-10 epochs), PPO achieves better sample efficiency than purely on-policy methods while maintaining stability.

**PPO Variants and Extensions (Optional)**

Since its introduction, PPO has spawned several important variants:

**PPO-Penalty**: Instead of clipping, this variant adds a KL penalty term:
$$L(\theta) = \mathbb{E}_t\left[r_t(\theta) A_t - \beta D_{KL}(\pi_{\theta_{\text{old}}}(\cdot|s_t) \| \pi_\theta(\cdot|s_t))\right]$$

**Adaptive PPO**: Dynamically adjusts the clipping parameter based on the observed KL divergence.

**PPO with Experience Replay**: Combines PPO with off-policy learning to improve sample efficiency further.

**PPO's Impact on RLHF**

PPO's efficiency and stability made large-scale RLHF feasible for the first time. Its adoption enabled:

- Training ChatGPT and GPT-4 with human feedback
- Scaling RLHF to models with hundreds of billions of parameters
- Making RLHF accessible to researchers and practitioners worldwide
- Establishing the standard pipeline for aligning language models

**Limitations and Challenges**

Despite its success, PPO has some limitations:

**1. Sample Efficiency**: While better than vanilla policy gradients, PPO still requires many samples compared to some off-policy methods.

**2. Hyperparameter Sensitivity**: Although robust, PPO's performance can still be sensitive to the choice of clipping parameter and other hyperparameters.

**3. Policy Collapse**: In some cases, PPO can lead to mode collapse where the policy becomes too deterministic.

**4. Distribution Mismatch**: Like all on-policy methods, PPO suffers when there's a significant mismatch between the data distribution and the current policy.

**Monotonic Improvement**: Under certain conditions, PPO can be shown to provide monotonic policy improvement.

**Sample Complexity**: PPO achieves $O(1/\epsilon)$ sample complexity for finding $\epsilon$-optimal policies in certain settings.

**Convergence**: With appropriate step sizes and clipping parameters, PPO converges to local optima of the policy gradient objective.

**PPO in Modern AI Systems**

Today, PPO remains the backbone of most large-scale RLHF systems:

- **OpenAI GPT series**: Used PPO for alignment training
- **Anthropic Claude**: Employs PPO-based techniques for constitutional AI
- **Google Bard/Gemini**: Utilizes PPO-style optimization for safety training
- **Open-source models**: Most open-source RLHF implementations use PPO

However, PPO's combination of simplicity, efficiency, and effectiveness has made it remarkably durable. Even as new methods emerge, PPO often serves as the baseline and continues to be competitive with more complex alternatives.

In our quantum computing education example, PPO would progressively refine our model through a series of efficient, clipped updates. Each update would cautiously improve the model's ability to engage children while avoiding the catastrophic policy changes that plagued earlier methods. The result would be a model that gradually learns to balance scientific accuracy with age-appropriate communication, creating explanations that are both correct and captivating.

PPO's genius lies not in mathematical sophistication but in practical insight: sometimes the best solution is the simplest one that actually works at scale. By replacing TRPO's complex machinery with elegant clipping, PPO democratized trust region methods and made stable policy optimization accessible to the entire machine learning community. This accessibility has been crucial for the rapid development and deployment of aligned AI systems, making PPO one of the most influential algorithms in modern reinforcement learning.

Let's start off by making a simple `PPOModel` class such that we can perform a `forward()` pass. 

In PPO, we refer to the policy model as the "actor" and the value model as the "critic".

The value model outputs scalar values (scores) just like the reward model. Meanwhile the policy model outputs probability distributions (take the log and you get logits).

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

from typing import Tuple
from transformers import Trainer

class PPOModel(nn.Module):
    def __init__(self, actor_model, critic_model):
        super().__init__()
        self.actor_model = actor_model
        self.critic_model = critic_model

    def forward(self, sequences, extra_inputs=None):
        # fetch logits from actor, fetch scalar reward from reference
        actor_logits = self.actor_model(**sequences, return_dict=True).logits
        critic_values = self.critic_model(**sequences)[-1]
        
        if extra_inputs is not None:
            extra_loss = self.actor_model(**extra_inputs, return_dict=True).loss
        else:
            extra_loss = 0.0
        return actor_logits, critic_values, extra_loss

Now, navigate to the `src/ppo` folder to find `ppo_trainer.py`. This will contain the main code for our PPO algorithm.

We will now go over each component of the `PPOTrainer` class.

The KL divergence cannot be computed in its original form. 

The action space (token space) is far too vast for us to sum/integrate over all $x$.
Furthermore, when we train, we don't store full probability distributions but rather log-probabilities of the tokens.
Thus, it doesn't make sense to waste GPU memory on something we can avoid. 
So, it is necessary to estimate it (with Monte Carlo, for example). However, we have a plethora of KL estimators to choose from.

Choosing the most "optimal" estimator is out of the scope of this notebook. We only implement the most popular estimators which have been tried and tested. 

Suppose $r=\frac{\pi (\theta)}{\pi_{\text{old}}(\theta)}$ is the ratio between the current policy and the old policy.
Let $k$ denote an ambiguous KL estimator. Then, we have the following:

$$k_3 = (r - 1) - \log r$$
$$k_{\text{abs}}=|\log r|$$
$$k_{\text{MSE}}=\frac{1}{2}(\log r)^2$$

This might be up for debate, but the best KL estimator is $k_{\text{MSE}}$. 

In [ ]:
def compute_rewards_with_kl_penalty(self, ref_values, actor_log_probs, ref_log_probs, responses_mask):
    """
    Computes rewards with the KL divergence penalty.
    Includes implementations of KL estimators since we can't compute it exactly.
    - k_3 is a popular KL estimator proposed here: http://joschu.net/blog/kl-approx.html

    Args:
        ref_values

        actor_log_probs: torch.Tensor
            log probabilities from our actor model

        ref_log_probs: torch.Tensor
            log probabilities from our reference model

        responses_mask: torch.Tensor
    """
    masks = responses_mask[:, 1:] 
    rewards_score = self.get_last_reward_score(ref_values, responses_mask)
    
    batch_size = rewards_score.shape[0]
    rewards_with_kl_penalty, kl_penalty_all = [], []

    for i in range(batch_size):
        mask = masks[i]: torch.Tensor
            shape: (bs, response_length)
        values: torch.Tensor
            
        eos_mask: torch.Tensor

        epsilon: torch.Tensor
    """
    
    # this keeps our value predictions within some epsilon-distance
    # mostly for numerical stability before performing regression
    clip_value_preds = torch.clamp(value_preds, values - epsilon, values + epsilon)   

    # thus, we have two errors, but it doesn't matter because we take the maximum (one)
    values_error = (value_preds - returns) ** 2
    clip_values_error = (clip_value_preds - returns) ** 2
    
    # this is essentially the inner sum for one trajectory t \in D_k where D_k is set of trajectories
    loss = 0.5 * masked_mean(torch
        lp_a = actor_lo: torch.Tensor
            shape: (bs, response_length)
        values: torch.Tensor
            
        eos_mask: torch.Tensor

        epsilon: torch.Tensor
    """
    
    # this keeps our value predictions within some epsilon-distance
    # mostly for numerical stability before performing regression
    clip_value_preds = torch.clamp(value_preds, values - epsilon, values + epsilon)   

    # thus, we have two errors, but it doesn't matter because we take the maximum (one)
    values_error = (value_preds - returns) ** 2
    clip_values_error = (clip_value_preds - returns) ** 2
    
    # this is essentially the inner sum for one trajectory t \in D_k where D_k is set of trajectories
    loss = 0.5 * masked_mean(torchg_probs[i][mask] # masked actor logprobs
        lp_r = ref_log_probs[i][mask] # masked reference logprobs


        # in my equations below, r is simply the ratio: pi(y) / pi_ref(y)
        if self.args.kl_penalty_method == 'k_3': # equation: (r - 1) - log r
            lp_diff = lp_a - lp_r
            ratio = torch.exp(lp_diff)
            kl_est = (ratio - 1.0) - lp_diff

        elif self.args.kl_penalty_method == 'abs': # equation: |log r|
            kl_est = torch.abs(lp_a - lp_r)

        elif self.args.kl_penalty_method == 'mse': # equation: 1/2 * (log r)^2
            kl_est = 0.5 * (lp_a - lp_r) ** 2 
        else:
            raise Value: torch.Tensor
            shape: (bs, response_length)
        values: torch.Tensor
            
        eos_mask: torch.Tensor

        epsilon: torch.Tensor
    """
    
    # this keeps our value predictions within some epsilon-distance
    # mostly for numerical stability before performing regression
    clip_value_preds = torch.clamp(value_preds, values - epsilon, values + epsilon)   

    # thus, we have two errors, but it doesn't matter because we take the maximum (one)
    values_error = (value_preds - returns) ** 2
    clip_values_error = (clip_value_preds - returns) ** 2
    
    # this is essentially the inner sum for one trajectory t \in D_k where D_k is set of trajectories
    loss = 0.5 * masked_mean(torchError(f"Unknown kl_penalty_method: {self.args.kl_penalty_method}")

            
        kl_penalty = - self.args.kl_penalty_beta * kl_est
        kl_penalty_all.append(kl_penalty)

        if self.args.reward_score_clip is not None:
            rewards_score[i] = torch.clamp(rewards_score[i], -self.args.reward_score_clip, self.args.reward_score_clip)
        
        end_index = mask.nonzero()[-1].detach().item()
        kl_penalty[end_index] += rewards_score[i]

        rewards_with_kl_penalty.append(kl_penalty)
    return torch.stack(rewards_with_kl_penalty), torch.stack(kl_penalty_all), rewards_score 

Now, we cannot compute the advantage function $A_t$ exactly, so we estimate it. 

The algorithm used in PPO to compute the estimate of the advantage $\hat{A}_t$ is Generalized Advantage Estimation (GAE).
One key thing here is that we do not just naively estimate $A_t$; we have to do it carefully to mitigate a large variance.

If $T$ is our trajectory length, then GAE computes this estimate by:

$$\hat{A}_t=\sum^{T-t-1}_{l=0}(\gamma\lambda)^l \delta_{t+l}$$

where $\gamma$ is the discount factor, $\lambda\in[0,1]$ is the GAE parameter, and $\delta_t=R(s_t,a_t)+\gamma V(s_{t+1})-V(s_t)$ which is the Temporal-Difference (TD) error. Here, $R(s_t, a_t)$ is the reward at time $t$, and $V(s_t)$ is the value function at time $t$. 

Variance reduction is a common theme in reinforcement learning as long-horizon estimates are prone to deviating from our expected value.
What GAE does is it performs multi-step "bootstrapping" to reduce the variance. Bootstrapping is a term which refers to updating an estimate value using (one or more) of the **same kind of estimate values**.

Without bootstrapping, our variance would explode, leading to longer episode trajectories (which we don't want). We want to keep these trajectories as short as possible; that is, to converge as fast as possible. In GAE, this bootstrapping can be seen in the TD error $\delta_t=R(s_t,a_t)+\gamma V(s_{t+1})-V(s_t)$.

However, we also need to consider bias. Remember the bias-variance tradeoff? Here, as we progress in the trajectory, our estimates accumulate positive bias at each step. This is the price to be paid for variance reduction at each step. There are proposed methods (such as VAPO https://arxiv.org/pdf/2504.05118) which aim to achieve both low variance while mitigating high bias. This is a bit more advanced, though.

In the context of applying this to language models, $T$ can be viewed as the maximum sequence length of a model's output. In GAE, the $t$-th step would be the $t$-th token that is currently being sampled from our model. The difference $T-t$ is just how far away this token is from the end of the sentence (EOS). The only nonzero reward $R(s_T)$ is calculated at the `<EOS>` token. Thus, all prior tokens are just propagating it backwards (with discounting). 

Given this discounting, as we increase the difference $T-t=2, T-t=3,..., T-t=k$, the reward signal grows weaker. Eventually, the last token to obtain this initial reward might obtain a zero value! Yikes. Thankfully, there are methods which attempt to address this decaying reward problem (such as VC-PPO https://arxiv.org/pdf/2503.01491).

In [ ]:
def compute_gae_advantage_return(self, rewards, values, mask, gamma, lam):
    """
    Computes the Generalized Advantage Estimation via Temporal-Difference with parameter lambda.

    Args:
        rewards: torch.Tensor
            shape: (bs, response_length)
        values: torch.Tensor
            shape: (bs, response_length)
        mask: torch.Tensor
            shape: (bs, response_length)
        gamma: float
            discount factor
        lam: float
            lambda parameter for GAE algorithm
    """
    B, T = rewards.shape # B is batch size, T is response_length

    # here, we bootstrap the value model updates with Temporal-Difference (parameter \lambda)
    with torch.no_grad():
        advantages_reversed = []
        lastgaelam = 0

        for t in reversed(range(T)):
            # for long sequences with T - t >> 1, discounting reduces the reward signal to near zero
            next_values = values[:, t + 1] if t < T - 1 else 0.0
            delta = rewards[:, t] + gamma * next_values - values[:, t]
            lastgaelam = (delta + gamma * lam * lastgaelam) * mask[:, t] 
            advantages_reversed.append(lastgaelam)
        advantages = torch.stack(advantages_reversed[::-1], dim=1)

        returns = advantages + values
        # 
        if use_advantage_norm:
            advantages = masked_whiten(advantages, eos_mask)
    return advantages, returns


Implementing PPO (which is an Actor-Critic algorithm) requires a lot of design choices, where you can easily lose yourself trying to read them all. However, the topmost choice deals with the actor and critic themselves. We have two choices: 

1. Learn two largely separate models. One for the actor and another for the critic.
2. Learn one large model, and apply two shallow heads at the end to format our output (one for the actor, another for the critic)

The first approach is usually more popular because the singular model we learn is usually a deep transformer which contains rich, hidden representations. As such, our two heads only need to be shallow because they serve as "mappings" from our representations to scalar values. The large transformer will have already done the grunt work during its intense training runs (and perhaps, any additional supervised fine-tuning).

In usual implementations of PPO (which is an Actor-Critic algorithm), we train a single large model with two heads on top. One is the actor (policy) head and the other is the critic (value) head.
We optimize this single model by minimizing a three-part loss function. 

That is, we add the $L^{CLIP}$ clipped surrogate loss as proposed in the original PPO paper. Next, we subtract an MSE error loss, $L^{VF}$ for computing the value function. Finally, we add the entropy of the policy, $H(\pi_{\theta}(\cdot | s))$ which is multiplied by some constant term, $c_1$. Usually, we also multiply $L^{VF}$ by a constant $c_2$ as well. So we have:
$$L=L^{CLIP}-c_2 L^{VF} + c_1 H(\pi_{\theta}(\cdot | s))$$

In the following sections, we will go over each term and their theoretical significances.

The actor has an entropy term which is effectively a regularizer that promotes exploration. Entropy in our case is (very loosely) a measure of our uncertainty in the policy. As such, it can tell us how confident a policy is at choosing an action given that they are in some state. When this entropy $H(\pi)$ is low, we have a high confidence, whereas a high $H(\pi)$ implies low confidence. 

We want our policy to diversify itself instead of picking the "best choice" each time. Thus, if we want to push the policy to "explore more", we need to keep some amount of uncertainty. We would effectively be "spreading" the probability distribution over multiple actions rather than letting a high confidence confine it to only a few actions. So, natually, we'd want to maximize the entropy $H(\pi)$. 

Another thing is that we don't always know how to compute $H(\pi)$ exactly, so we estimate it by taking the log-probability of our policy and averaging over the batch. If $\mathcal{B}$ is our batch of state, action pairs, then we have:
$$H(\pi)\approx -\frac{1}{\mathcal{|B|}}\sum_{a,s\in\mathcal{B}} (-\log \pi(a|s))=\frac{1}{\mathcal{|B|}}\sum_{a,s\in\mathcal{B}} (\log \pi(a|s))$$
Here, the negative in the front is by the definition of entropy $H(x)=-\sum_x p(x)\log(p(x))$, but since we are estimating it we can omit the $p(x)$ term so we have something like $H(x)\approx -\sum_x \log(p(x))$. The negative in front of $\log \pi(a|s)$ is because we are collecting negative log-probabilities. It is usually easier to compute log-probabilities than the full probabilities, hence this estimation. 

So, our entropy $H(\pi)$ comes out as positive. Furthermore, we add a constant $c_1$ which we can tweak to regularize the effect of $H(\pi)$ on the overall loss $L$.
$$c_1 H(\pi(\cdot|s))$$
However, we can afford computing the entropy in its entire form (which is what the code does below):
$$H(\pi)= -\frac{1}{\mathcal{|B|}}\sum_{a,s\in\mathcal{B}} (-\pi(a|s)\log \pi(a|s))=\frac{1}{\mathcal{|B|}}\sum_{a,s\in\mathcal{B}} \pi(a|s)\log \pi(a|s)$$

In [ ]:
def get_entropy(self, logits, mask):
        """
        Computes the entropy of the policy, which incentivizes exploration.

        Args:
            logits: torch.Tensor
                unnormalized log-probabilities from a model
            mask: torch.Tensor
                mask to be applied to the computed entropy
        """
        probs = F.softmax(logits, dim=-1)
        log_probs = F.log_softmax(logits, dim=-1)
        entropy = self.masked_mean(-torch.sum(probs * log_probs, dim=-1), mask)
        return entropy

The PPO-Clip surrogate objective is defined as:
$$ L^{\text{CLIP}} = \min\left(\frac{\pi(\theta)}{\pi_{\text{old}}(\theta)}A_t, \text{clip}\left( \frac{\pi(\theta)}{\pi_{\text{old}}(\theta)} , 1-\epsilon, 1+\epsilon \right)A_t \right) $$

Notice how we do not have a `min` operation here? Notice how we actually use `torch.max()`? That seems counterintuitive. However, further note that we added a negative sign to `loss1` and `loss2`, before taking their `max`. 

This is effectively equivalent to the `min()` operation but done in reverse to ensure numerical stability. 

In [ ]:
def compute_policy_loss(self, old_log_prob, log_prob, advantages, eos_mask, epsilon):
        """
        Computes the policy gradient loss function for PPO.

        Args:
            old_log_prob: torch.Tensor
                log probabilities from the old policy
            log_prob: torch.Tensor
                log probabilities from the current policy 
            advantages: torch.Tensor
                Computed advantages via advantage estimation
            eos_mask: torch.Tensor
            
            epsilon: float
        """

        # from log domain -> real domain
        ratio = torch.exp(log_prob - old_log_prob)
        loss1 = -advantages * ratio
        loss2 = -advantages * torch.clamp(ratio, 1.0 - epsilon, 1.0 + epsilon)

        loss = masked_mean(torch.max(loss1, loss2), eos_mask)

As detailed in PPO-Clip algorithm, we compute the value function by regressing on an MSE loss. 

For some reason this is not explicitly mentioned in the original PPO paper. It is, however, mentioned in OpenAI's Spinning Up documentation entry on PPO. For the answer, we need to do some digging back to the old Barto & Sutton book (you can not escape RL!!!)

Recall that the on-policy value function is defined as the expected discounted return that the agent can get given he starts in some state $s$ and acts according to the policy $\pi$:
$$v_{\pi}(s)=\mathbb{E}_{\pi}[R_{t+1}+\gamma R_{t+2}+\cdots |S_t=s]$$
Now, define the discounted return as $G_t=\sum_{t'=t}^T \gamma^{t'-t}r_{t'}$. If we reindex with $k=t'-t$ then this becomes $G_t=\sum_{k=0}^T \gamma^{k}r_{t}$.
So, the value function becomes:
$$v_{\pi}(s)=\mathbb{E}_{\pi}[G_t|S_t=s]=\mathbb{E}_{\pi}[\sum_{k=0}^T \gamma^{k}r_{t}|S_t=s]$$
which makes a lot more sense now! We see that $v_{\pi}(s)$ is just the expected discounted return! 


It is found in Chapter 9.2 of the book. Here, the "natural objective function" for value-based algorithms is to minimize the MSE between the true value and observed returns.
That is, our value error is defined as:
$$\overline{VE}(\mathbf{w})=\mathbb{E}[v_{\pi}(s)-\hat{v}(s,\mathbf{w})]^2$$
where the expectation $\mathbb{E}[\cdot]$ is taken over the state distribution $\mu(s)$ such that $\mu(s)\geq 0$ and $\sum_s \mu(s)=1$.

Dissecting this equation, we can see that the expected discounted return $v_{\pi}(s)$ can be interpreted as the empirical return $G_t$. However, this is not the exactly the $G_t$ we defined earlier. In practice, we do not compute the full return, so we usually use something like a Monte Carlo estimate. It can be shown that this Monte Carlo estimate is an unbiased sample of $v_{\pi}(s)$, so they are not quite equal (but we can get close enough).

Next, $\hat{v}(s, \mathbf{w})$ is defined as a joint distribution with states $s$ and weights $\mathbf{w}$ (which we learn). This is effectively our learned value function, $V(s_t)$ which can be computed using a multi-layer neural network. 

Thus, in our PPO-Clip algorithm we have the MSE loss:
$$L_v = \sum_{t=1}^T \frac{1}{2}(G_t - V(S_t))^2$$
which tells us how far our observed returns (value estimates), $V(s_t)$, deviate from the "true" empirical returns, $G_t$.

For a further dissection of On-Policy Prediction with Approximation, I recommend reading Chapter 9 of Sutton & Barto.

In [ ]:
def compute_value_loss(self, value_preds, returns, values, eos_mask, epsilon):
    """
    Fits the value function by regression on the MSE loss. 

    Args:
        value_preds: torch.Tensor
            the predictions from our value model
        returns: torch.Tensor
            shape: (bs, response_length)
        values: torch.Tensor
            
        eos_mask: torch.Tensor

        epsilon: torch.Tensor
    """
    
    # this keeps our value predictions within some epsilon-distance
    # mostly for numerical stability before performing regression
    clip_value_preds = torch.clamp(value_preds, values - epsilon, values + epsilon)   

    # thus, we have two errors, but it doesn't matter because we take the maximum (one)
    values_error = (value_preds - returns) ** 2
    clip_values_error = (clip_value_preds - returns) ** 2
    
    # this is essentially the inner sum for one trajectory t \in D_k where D_k is set of trajectories
    loss = 0.5 * masked_mean(torch.max(values_error, clip_values_error), eos_mask)
    return loss, values_error

Notice that in the above code cell, we called `masked_mean()`? What is this? Well, we're now getting into implementation-specific "tricks" associated with PPO. Stuff you might not find in the original literature.



<img src="assets/grpo.png">

Now in this one, explain a lot, what it is, use our quantum 5 year old analogies how it works laydown the components and how it was groundbreaking, dive into the math side of it a lot we wont be doing code here since we're mainly focusing on PPO and DPO for this notebook

#### Direct Preference Optimization

Direct Preference Optimization (DPO), introduced by Rafailov et al. in 2023, represents perhaps the most revolutionary breakthrough in RLHF since the field's inception. While PPO and other reinforcement learning methods transformed language model alignment, DPO achieved something that seemed almost impossible: eliminating the need for reinforcement learning entirely while maintaining the same theoretical guarantees and often achieving superior practical performance.

To understand DPO's groundbreaking impact, let's return to our quantum computing explanation scenario and see how it completely reimagines the alignment process by directly optimizing the language model on human preferences without ever training a separate reward model or using complex RL algorithms.

Before diving into DPO's mechanics, we need to understand the fundamental question that motivated its development: "What if the entire RLHF pipeline - reward models, value functions, policy gradients, and complex optimization procedures - is unnecessarily complicated?" 

This might seem heretical given how much effort the field has invested in perfecting RL-based approaches, but DPO's creators asked a deeper question: "Can we directly optimize language models on preference data without the intermediate steps?"

Consider our traditional RLHF workflow:
1. **SFT**: Train on demonstrations → Get decent responses
2. **Preference Collection**: Humans compare responses → Get preference pairs  
3. **Reward Model**: Train neural network to predict preferences → Get automated evaluator
4. **RL Optimization**: Use PPO to maximize rewards → Get aligned model

DPO's insight was that steps 3 and 4 might be redundant. If we have preference data directly from humans, why not optimize the language model on that data directly? This seemingly simple question led to one of the most elegant mathematical reformulations in modern machine learning.

##### The Mathematical Breakthrough: Reparameterizing the Reward Model

DPO's theoretical foundation rests on a brilliant mathematical insight about the relationship between optimal policies and reward functions. Let's build this up step by step, starting with what we know from traditional RLHF.

**Traditional RLHF Objective:**
In PPO-style RLHF, we try to find the policy $\pi_\theta$ that maximizes expected reward while staying close to a reference policy $\pi_{\text{ref}}$:

$$\pi^* = \arg\max_\pi \mathbb{E}_{x \sim \mathcal{D}, y \sim \pi(y|x)}[r(x,y)] - \beta D_{KL}(\pi(y|x) \| \pi_{\text{ref}}(y|x))$$

where:
- $r(x,y)$ is our reward model's score for response $y$ to prompt $x$
- $\beta$ controls how much we penalize deviation from the reference policy
- $D_{KL}$ is the KL divergence preventing the model from changing too dramatically

This is a constrained optimization problem that PPO solves through complex policy gradients.

DPO's breakthrough was recognizing that this constrained optimization problem has a closed-form solution! Using the method of Lagrange multipliers, we can solve for the optimal policy directly:

$$\pi^*(y|x) = \frac{1}{Z(x)} \pi_{\text{ref}}(y|x) \exp\left(\frac{1}{\beta} r(x,y)\right)$$

where $Z(x) = \sum_y \pi_{\text{ref}}(y|x) \exp\left(\frac{1}{\beta} r(x,y)\right)$ is the partition function.

The partition function $Z(x)$ is a normalization term that ensures the probability distribution $\pi^*(y|x)$ sums to 1 across all possible responses $y$ for a given prompt $x$. It's a fundamental concept in statistical physics and probability theory, essentially accounting for all possible states of the system. In this context, it aggregates the weighted probabilities of all possible responses, with each response weighted by the exponential of its reward. This normalization is crucial for maintaining a valid probability distribution when transforming between reward functions and policies.

This equation tells us exactly what the optimal policy looks like in terms of the reward function and reference policy. But here's the crucial insight: we can rearrange this equation to solve for the reward function in terms of the optimal policy:

$$r(x,y) = \beta \log \frac{\pi^*(y|x)}{\pi_{\text{ref}}(y|x)} + \beta \log Z(x)$$

Since $Z(x)$ depends only on the prompt $x$ (not the response $y$), it cancels out when we compute preference probabilities. This leads to DPO's fundamental insight:

$$r(x,y) = \beta \log \frac{\pi_\theta(y|x)}{\pi_{\text{ref}}(y|x)} + \text{constant}$$

This means we can express any reward function implicitly through the ratio between our policy and a reference policy! This is the mathematical foundation that makes DPO possible.

##### From Reward Models to Direct Optimization

Now let's see how this reparameterization transforms the entire RLHF process. Remember our Bradley-Terry preference model from earlier:

$$P(y_w \succ y_l | x) = \sigma(r(x, y_w) - r(x, y_l))$$

Substituting our DPO reparameterization:

$$P(y_w \succ y_l | x) = \sigma\left(\beta \log \frac{\pi_\theta(y_w|x)}{\pi_{\text{ref}}(y_w|x)} - \beta \log \frac{\pi_\theta(y_l|x)}{\pi_{\text{ref}}(y_l|x)}\right)$$

This simplifies beautifully to:

$$P(y_w \succ y_l | x) = \sigma\left(\beta \log \frac{\pi_\theta(y_w|x)}{\pi_{\text{ref}}(y_w|x)} - \beta \log \frac{\pi_\theta(y_l|x)}{\pi_{\text{ref}}(y_l|x)}\right)$$

This gives us DPO's training objective - the same Bradley-Terry loss we used for reward model training, but now directly in terms of the language model probabilities:

$$\mathcal{L}_{\text{DPO}}(\pi_\theta) = -\mathbb{E}_{(x,y_w,y_l) \sim \mathcal{D}} \left[ \log \sigma\left(\beta \log \frac{\pi_\theta(y_w|x)}{\pi_{\text{ref}}(y_w|x)} - \beta \log \frac{\pi_\theta(y_l|x)}{\pi_{\text{ref}}(y_l|x)}\right) \right]$$

This is mathematically equivalent to the traditional RLHF objective, but it can be optimized directly without reward models or RL algorithms!



##### Understanding the Reference Policy

The reference policy $\pi_{\text{ref}}$ plays a crucial role in DPO that's worth understanding deeply:

**What is the Reference Policy?**
The reference policy is typically the SFT model - our starting point before preference optimization. It serves as an "anchor" that prevents the model from changing too dramatically during training.

**Why Do We Need a Reference Policy?**
Without the reference policy, the DPO objective becomes ill-defined. The reference policy provides:
1. **Stability**: Prevents the model from collapsing to degenerate solutions
2. **Regularization**: Maintains the model's general capabilities while improving alignment
3. **Baseline**: Defines what "change" means in the context of preference optimization

**Mathematical Role**:
The reference policy appears in the ratio $\frac{\pi_\theta(y|x)}{\pi_{\text{ref}}(y|x)}$, which measures how much the current policy deviates from the starting point. This ratio:
- Increases for responses the model is learning to prefer
- Decreases for responses the model is learning to avoid
- Stays near 1.0 for responses that weren't in the training data

For our quantum explanation example:
- $\frac{\pi_\theta(\text{"Hey there! Imagine..."}))}{\pi_{\text{ref}}(\text{"Hey there! Imagine..."})} > 1$ (model learned to prefer engaging openings)
- $\frac{\pi_\theta(\text{"Quantum superposition involves..."}))}{\pi_{\text{ref}}(\text{"Quantum superposition involves..."})} < 1$ (model learned to avoid technical jargon)

##### Why This Works?

Let's understand what this loss function actually does using our quantum computing explanation example:

**Scenario**: We have preference data showing humans prefer Response B over Response A:
- **Response A**: "Quantum computers use qubits that exist in superposition states..."
- **Response B**: "Hey there! Imagine quantum computers as magical calculators that can solve puzzles by being in multiple places at once!"

**Traditional RLHF Process**:
1. Train reward model: $r(x, \text{Response B}) > r(x, \text{Response A})$
2. Use PPO to increase probability of higher-reward responses
3. Model learns: $\pi_\theta(\text{Response B}|x) \uparrow$, $\pi_\theta(\text{Response A}|x) \downarrow$

**DPO Process**:
1. Direct optimization: $\frac{\pi_\theta(\text{Response B}|x)}{\pi_{\text{ref}}(\text{Response B}|x)} > \frac{\pi_\theta(\text{Response A}|x)}{\pi_{\text{ref}}(\text{Response A}|x)}$
2. Model learns to assign higher relative probability to preferred responses
3. Same end result: $\pi_\theta(\text{Response B}|x) \uparrow$, $\pi_\theta(\text{Response A}|x) \downarrow$

The key insight is that DPO directly optimizes the probability ratios that determine human preferences, skipping the intermediate reward model entirely.


<img src="assets/dpo.png">

Let's trace through DPO's streamlined pipeline compared to traditional RLHF:

**Step 1: Start with SFT Model (Same as RLHF)**
We begin with a supervised fine-tuned model that can follow basic instructions. This serves as both our starting point for optimization and our reference policy $\pi_{\text{ref}}$.

**Step 2: Collect Preference Data (Same as RLHF)**  
Human annotators compare pairs of responses and indicate preferences. For our quantum explanation:
- Prompt: "Explain quantum computing to a 10-year-old"
- Response pair comparison → Human prefers engaging, analogical explanation

**Step 3: Direct Preference Optimization (Revolutionary Change)**
Instead of training a reward model, we directly optimize the language model using the DPO loss:

For each preference pair $(x, y_w, y_l)$:
1. **Compute probabilities**: 
   - $\pi_\theta(y_w|x)$ and $\pi_\theta(y_l|x)$ from current model
   - $\pi_{\text{ref}}(y_w|x)$ and $\pi_{\text{ref}}(y_l|x)$ from reference (SFT) model

2. **Calculate preference probability**:
   $$P(\text{prefer } y_w) = \sigma\left(\beta \log \frac{\pi_\theta(y_w|x)/\pi_{\text{ref}}(y_w|x)}{\pi_\theta(y_l|x)/\pi_{\text{ref}}(y_l|x)}\right)$$

3. **Optimize to match human preferences**:
   - If humans preferred $y_w$, increase this probability
   - Adjust model parameters to make preferred responses more likely

**Step 4: Iterate Until Convergence**
Repeat the optimization process until the model consistently generates responses that align with human preferences.



##### Why DPO is Theoretically Equivalent to RLHF

One of DPO's most remarkable properties is that it's not just similar to traditional RLHF - it's mathematically equivalent under certain conditions. Let's understand why:

**Equivalence Theorem**: If we solve the DPO optimization problem to convergence, the resulting policy $\pi^*_{\text{DPO}}$ is identical to the policy we would get from solving the constrained RLHF problem:

$$\pi^*_{\text{DPO}} = \pi^*_{\text{RLHF}} = \arg\max_\pi \mathbb{E}[r(x,y)] - \beta D_{KL}(\pi \| \pi_{\text{ref}})$$

**Proof Sketch**: 
The key insight is that the DPO loss function is the gradient of the RLHF objective with respect to the policy parameters. When we minimize the DPO loss, we're following the exact same gradient flow that RLHF algorithms like PPO approximate.

This theoretical equivalence is profound because it means DPO isn't making any compromises - it achieves the same optimal solution as traditional RLHF but through a much simpler path.

##### Why DPO is More Efficient

DPO's computational benefits become apparent when we compare the training processes:

**Traditional RLHF Computational Complexity**:
- **Reward Model Training**: Forward/backward passes through full transformer
- **PPO Optimization**: 
  - Multiple rollout generations per iteration
  - Value function estimation and advantage computation  
  - Multiple policy gradient steps per batch
  - KL divergence computations
- **Total**: ~3-5x computational cost of supervised training

**DPO Computational Complexity**:
- **Direct Optimization**: Standard gradient descent on preference data
- **Two Forward Passes**: Current model + reference model probability computation
- **Simple Loss**: Bradley-Terry preference loss (just like reward model training)
- **Total**: ~1.2x computational cost of supervised training

For our quantum explanation model, this means:
- **RLHF**: Might require 100 GPU-hours for alignment
- **DPO**: Achieves same results in ~30 GPU-hours

This efficiency gain is crucial for making alignment accessible to researchers and practitioners without massive computational resources.

##### DPO's Practical Advantages: Beyond Computational Efficiency

**1. Simplicity and Reliability**
DPO eliminates many sources of instability in traditional RLHF:
- No reward model overoptimization
- No complex PPO hyperparameter tuning  
- No value function estimation errors
- Standard supervised learning dynamics

**2. Better Sample Efficiency**
DPO often achieves better alignment with fewer preference pairs because:
- Direct optimization on preference data (no information loss through reward modeling)
- No policy-reward model mismatch issues
- More stable training dynamics

**3. Easier Hyperparameter Tuning**
DPO has fewer critical hyperparameters:
- $\beta$: Controls the strength of the KL penalty (typical values: 0.1-0.5)
- Learning rate: Standard supervised learning values work well
- No need for PPO-specific hyperparameters (clipping, GAE, etc.)

**4. Interpretability**
The DPO loss is more interpretable:
- Direct relationship between loss and human preferences
- Easy to debug and understand training dynamics
- Clear connection between loss values and model behavior

##### When DPO Might Struggle: Understanding the Limitations

Despite its advantages, DPO has some limitations worth understanding:

**1. Limited Exploration**
DPO can only learn from the preference data provided. If the preference dataset has gaps, DPO might not discover better responses that weren't in the training data. Traditional RLHF with RL exploration might find better solutions in some cases.

**2. Preference Dataset Quality Dependency**
DPO's performance is directly tied to the quality of preference data. Poor or inconsistent human preferences will directly impact the final model quality, whereas RLHF's reward model might provide some smoothing of noisy preferences.

**3. Distributional Mismatch**
If the distribution of prompts in preference data differs significantly from deployment, DPO might not generalize as well as RLHF methods that learn more general reward functions.

##### DPO Variants and Extensions

Since its introduction, several variants of DPO have emerged to address specific limitations:

**1. Self-Play DPO (SPDPO)**
Iteratively generates new response pairs from the current model and collects preferences, addressing the exploration limitation.

**2. Constitutional DPO (CDPO)**  
Uses AI-generated critiques and revisions instead of human preferences, making the process more scalable.

**3. Rejection Sampling DPO (RSDPO)**
Combines DPO with rejection sampling to improve sample quality before training.

**4. Online DPO**
Collects preferences on-the-fly during training, similar to online RLHF methods.

##### DPO in Practice: Real-World Impact

DPO has been rapidly adopted across the AI industry:

**Major Implementations**:
- **Anthropic**: Uses DPO variants in Claude training
- **Meta**: Llama 2-Chat uses DPO-style optimization
- **Mistral**: Mistral-Instruct employs DPO techniques
- **Open-source**: Most new instruction-tuned models use DPO

**Performance Results**:
Studies consistently show DPO achieving comparable or superior performance to PPO-based RLHF while being significantly more efficient and stable.

##### Is DPO the End of RLHF?

DPO's success raises important questions about the future of AI alignment:

**Arguments for DPO Dominance**:
- Simpler, more stable, more efficient
- Mathematically equivalent to RLHF under ideal conditions
- Easier to implement and debug
- Better resource utilization

**Arguments for Continued RLHF Development**:
- RL methods might discover better solutions through exploration
- Complex multi-step reasoning might benefit from reward shaping
- Online learning and adaptation scenarios favor RL approaches
- Some alignment objectives might be easier to express as rewards

**The Likely Future**:
Rather than complete replacement, we're likely to see:
- DPO as the default for most applications
- RLHF reserved for specialized scenarios requiring exploration
- Hybrid approaches combining the best of both methods
- Continued theoretical work unifying the approaches


In our quantum computing education example, DPO would directly learn from human preferences to transform our model from generating technical responses to producing engaging, child-friendly explanations. The process would be simpler, faster, and more stable than traditional RLHF while achieving the same high-quality results.

DPO represents a paradigm shift in AI alignment - proof that sometimes the most elegant solution is also the most practical. By eliminating unnecessary complexity while maintaining theoretical rigor, DPO has democratized high-quality alignment and opened new possibilities for building AI systems that better serve human needs and values.

#### Group Relative Policy Optimization

## Citations